# 第九章：元编程
 软件开发领域中最经典的口头禅就是“don’t repeat yourself”。
也就是说，任何时候当你的程序中存在高度重复(或者是通过剪切复制)的代码时，都应该想想是否有更好的解决方案。
在Python当中，通常都可以通过元编程来解决这类问题。
简而言之，元编程就是关于创建操作源代码(比如修改、生成或包装原来的代码)的函数和类。
主要技术是使用装饰器、类装饰器和元类。不过还有一些其他技术，
包括签名对象、使用 exec() 执行代码以及对内部函数和类的反射技术等。
本章的主要目的是向大家介绍这些元编程技术，并且给出实例来演示它们是怎样定制化你的源代码行为的。

## 9.1 在函数上添加包装器


### 问题


你想在函数上添加一个包装器，增加额外的操作处理(比如日志、计时等)。

### 解决方案


如果你想使用额外的代码包装一个函数，可以定义一个装饰器函数，例如：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




d
e
f
 
t
i
m
e
t
h
i
s
(
f
u
n
c
)
:


 
 
 
 
'
'
'


 
 
 
 
D
e
c
o
r
a
t
o
r
 
t
h
a
t
 
r
e
p
o
r
t
s
 
t
h
e
 
e
x
e
c
u
t
i
o
n
 
t
i
m
e
.


 
 
 
 
'
'
'


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
t
a
r
t
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
r
e
s
u
l
t
 
=
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
e
n
d
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
u
n
c
.
_
_
n
a
m
e
_
_
,
 
e
n
d
-
s
t
a
r
t
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r



下面是使用装饰器的例子：

In [ ]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
countdown(100000)

In [ ]:
countdown(10000000)

### 讨论


一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数。
当你像下面这样写：

In [ ]:
@
t
i
m
e
t
h
i
s


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
p
a
s
s



跟像下面这样写其实效果是一样的：

In [ ]:
d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
p
a
s
s


c
o
u
n
t
d
o
w
n
 
=
 
t
i
m
e
t
h
i
s
(
c
o
u
n
t
d
o
w
n
)



顺便说一下，内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。
例如，下面这两个代码片段是等价的：

In [ ]:
c
l
a
s
s
 
A
:


 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
m
e
t
h
o
d
(
c
l
s
)
:


 
 
 
 
 
 
 
 
p
a
s
s




c
l
a
s
s
 
B
:


 
 
 
 
#
 
E
q
u
i
v
a
l
e
n
t
 
d
e
f
i
n
i
t
i
o
n
 
o
f
 
a
 
c
l
a
s
s
 
m
e
t
h
o
d


 
 
 
 
d
e
f
 
m
e
t
h
o
d
(
c
l
s
)
:


 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
m
e
t
h
o
d
 
=
 
c
l
a
s
s
m
e
t
h
o
d
(
m
e
t
h
o
d
)



在上面的 wrapper() 函数中，
装饰器内部定义了一个使用 *args 和  **kwargs 来接受任意参数的函数。
在这个函数里面调用了原始函数并将其结果返回，不过你还可以添加其他额外的代码(比如计时)。
然后这个新的函数包装器被作为结果返回来代替原始函数。

需要强调的是装饰器并不会修改原始函数的参数签名以及返回值。
使用 *args 和  **kwargs 目的就是确保任何参数都能适用。
而返回结果值基本都是调用原始函数 func(*args, **kwargs) 的返回结果，其中func就是原始函数。

刚开始学习装饰器的时候，会使用一些简单的例子来说明，比如上面演示的这个。
不过实际场景使用时，还是有一些细节问题要注意的。
比如上面使用 @wraps(func) 注解是很重要的，
它能保留原始函数的元数据(下一小节会讲到)，新手经常会忽略这个细节。
接下来的几个小节我们会更加深入的讲解装饰器函数的细节问题，如果你想构造你自己的装饰器函数，需要认真看一下。

## 9.2 创建装饰器时保留函数元信息


### 问题


你写了一个装饰器作用在某个函数上，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。

### 解决方案


任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数。例如：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s


d
e
f
 
t
i
m
e
t
h
i
s
(
f
u
n
c
)
:


 
 
 
 
'
'
'


 
 
 
 
D
e
c
o
r
a
t
o
r
 
t
h
a
t
 
r
e
p
o
r
t
s
 
t
h
e
 
e
x
e
c
u
t
i
o
n
 
t
i
m
e
.


 
 
 
 
'
'
'


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
t
a
r
t
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
r
e
s
u
l
t
 
=
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
e
n
d
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
f
u
n
c
.
_
_
n
a
m
e
_
_
,
 
e
n
d
-
s
t
a
r
t
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
r
e
s
u
l
t


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r



下面我们使用这个被包装后的函数并检查它的元信息：

In [ ]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
countdown(100000)

In [ ]:
countdown.__name__

In [ ]:
countdown.__doc__

In [ ]:
countdown.__annotations__

### 讨论


在编写装饰器的时候复制元信息是一个非常重要的部分。如果你忘记了使用 @wraps ，
那么你会发现被装饰函数丢失了所有有用的信息。比如如果忽略 @wraps 后的效果是下面这样的：

In [ ]:
countdown.__name__

In [ ]:
countdown.__doc__
countdown.__annotations__

@wraps 有一个重要特征是它能让你通过属性 __wrapped__ 直接访问被包装函数。例如:

In [ ]:
countdown.__wrapped__(100000)

__wrapped__ 属性还能让被装饰函数正确暴露底层的参数签名信息。例如：

In [ ]:
from inspect import signature
print(signature(countdown))

一个很普遍的问题是怎样让装饰器去直接复制原始函数的参数签名信息，
如果想自己手动实现的话需要做大量的工作，最好就简单的使用 @wraps 装饰器。
通过底层的 __wrapped__ 属性访问到函数签名信息。更多关于签名的内容可以参考9.16小节。

## 9.3 解除一个装饰器


### 问题


一个装饰器已经作用在一个函数上，你想撤销它，直接访问原始的未包装的那个函数。

### 解决方案


假设装饰器是通过 @wraps (参考9.2小节)来实现的，那么你可以通过访问 __wrapped__ 属性来访问原始函数：

In [ ]:
@somedecorator
def add(x, y):
    return x + y
orig_add = add.__wrapped__
orig_add(3, 4)

### 讨论


直接访问未包装的原始函数在调试、内省和其他函数操作时是很有用的。
但是我们这里的方案仅仅适用于在包装器中正确使用了 @wraps 或者直接设置了 __wrapped__ 属性的情况。

如果有多个包装器，那么访问 __wrapped__ 属性的行为是不可预知的，应该避免这样做。
在Python3.3中，它会略过所有的包装层，比如，假如你有如下的代码：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




d
e
f
 
d
e
c
o
r
a
t
o
r
1
(
f
u
n
c
)
:


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
D
e
c
o
r
a
t
o
r
 
1
'
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




d
e
f
 
d
e
c
o
r
a
t
o
r
2
(
f
u
n
c
)
:


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
D
e
c
o
r
a
t
o
r
 
2
'
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




@
d
e
c
o
r
a
t
o
r
1


@
d
e
c
o
r
a
t
o
r
2


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y



下面我们在Python3.3下测试：

In [ ]:
add(2, 3)

In [ ]:
add.__wrapped__(2, 3)

下面我们在Python3.4下测试：

In [ ]:
add(2, 3)

In [ ]:
add.__wrapped__(2, 3)

最后要说的是，并不是所有的装饰器都使用了 @wraps ，因此这里的方案并不全部适用。
特别的，内置的装饰器 @staticmethod 和 @classmethod 就没有遵循这个约定
(它们把原始函数存储在属性 __func__ 中)。

## 9.4 定义一个带参数的装饰器


### 问题


你想定义一个可以接受参数的装饰器

### 解决方案


我们用一个例子详细阐述下接受参数的处理过程。
假设你想写一个装饰器，给函数添加日志功能，同时允许用户指定日志的级别和其他的选项。
下面是这个装饰器的定义和使用示例：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s


i
m
p
o
r
t
 
l
o
g
g
i
n
g




d
e
f
 
l
o
g
g
e
d
(
l
e
v
e
l
,
 
n
a
m
e
=
N
o
n
e
,
 
m
e
s
s
a
g
e
=
N
o
n
e
)
:


 
 
 
 
"
"
"


 
 
 
 
A
d
d
 
l
o
g
g
i
n
g
 
t
o
 
a
 
f
u
n
c
t
i
o
n
.
 
l
e
v
e
l
 
i
s
 
t
h
e
 
l
o
g
g
i
n
g


 
 
 
 
l
e
v
e
l
,
 
n
a
m
e
 
i
s
 
t
h
e
 
l
o
g
g
e
r
 
n
a
m
e
,
 
a
n
d
 
m
e
s
s
a
g
e
 
i
s
 
t
h
e


 
 
 
 
l
o
g
 
m
e
s
s
a
g
e
.
 
I
f
 
n
a
m
e
 
a
n
d
 
m
e
s
s
a
g
e
 
a
r
e
n
'
t
 
s
p
e
c
i
f
i
e
d
,


 
 
 
 
t
h
e
y
 
d
e
f
a
u
l
t
 
t
o
 
t
h
e
 
f
u
n
c
t
i
o
n
'
s
 
m
o
d
u
l
e
 
a
n
d
 
n
a
m
e
.


 
 
 
 
"
"
"


 
 
 
 
d
e
f
 
d
e
c
o
r
a
t
e
(
f
u
n
c
)
:


 
 
 
 
 
 
 
 
l
o
g
n
a
m
e
 
=
 
n
a
m
e
 
i
f
 
n
a
m
e
 
e
l
s
e
 
f
u
n
c
.
_
_
m
o
d
u
l
e
_
_


 
 
 
 
 
 
 
 
l
o
g
 
=
 
l
o
g
g
i
n
g
.
g
e
t
L
o
g
g
e
r
(
l
o
g
n
a
m
e
)


 
 
 
 
 
 
 
 
l
o
g
m
s
g
 
=
 
m
e
s
s
a
g
e
 
i
f
 
m
e
s
s
a
g
e
 
e
l
s
e
 
f
u
n
c
.
_
_
n
a
m
e
_
_




 
 
 
 
 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
l
o
g
.
l
o
g
(
l
e
v
e
l
,
 
l
o
g
m
s
g
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r


 
 
 
 
r
e
t
u
r
n
 
d
e
c
o
r
a
t
e




#
 
E
x
a
m
p
l
e
 
u
s
e


@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
D
E
B
U
G
)


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y




@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
C
R
I
T
I
C
A
L
,
 
'
e
x
a
m
p
l
e
'
)


d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
!
'
)



初看起来，这种实现看上去很复杂，但是核心思想很简单。
最外层的函数 logged() 接受参数并将它们作用在内部的装饰器函数上面。
内层的函数 decorate() 接受一个函数作为参数，然后在函数上面放置一个包装器。
这里的关键点是包装器是可以使用传递给 logged() 的参数的。

### 讨论


定义一个接受参数的包装器看上去比较复杂主要是因为底层的调用序列。特别的，如果你有下面这个代码：

In [ ]:
@
d
e
c
o
r
a
t
o
r
(
x
,
 
y
,
 
z
)


d
e
f
 
f
u
n
c
(
a
,
 
b
)
:


 
 
 
 
p
a
s
s



装饰器处理过程跟下面的调用是等效的;

In [ ]:
d
e
f
 
f
u
n
c
(
a
,
 
b
)
:


 
 
 
 
p
a
s
s


f
u
n
c
 
=
 
d
e
c
o
r
a
t
o
r
(
x
,
 
y
,
 
z
)
(
f
u
n
c
)



decorator(x, y, z) 的返回结果必须是一个可调用对象，它接受一个函数作为参数并包装它，
可以参考9.7小节中另外一个可接受参数的包装器例子。

## 9.5 可自定义属性的装饰器


### 问题


你想写一个装饰器来包装一个函数，并且允许用户提供参数在运行时控制装饰器行为。

### 解决方案


引入一个访问函数，使用 nonlocal 来修改内部变量。
然后这个访问函数被作为一个属性赋值给包装函数。

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s
,
 
p
a
r
t
i
a
l


i
m
p
o
r
t
 
l
o
g
g
i
n
g


#
 
U
t
i
l
i
t
y
 
d
e
c
o
r
a
t
o
r
 
t
o
 
a
t
t
a
c
h
 
a
 
f
u
n
c
t
i
o
n
 
a
s
 
a
n
 
a
t
t
r
i
b
u
t
e
 
o
f
 
o
b
j


d
e
f
 
a
t
t
a
c
h
_
w
r
a
p
p
e
r
(
o
b
j
,
 
f
u
n
c
=
N
o
n
e
)
:


 
 
 
 
i
f
 
f
u
n
c
 
i
s
 
N
o
n
e
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
p
a
r
t
i
a
l
(
a
t
t
a
c
h
_
w
r
a
p
p
e
r
,
 
o
b
j
)


 
 
 
 
s
e
t
a
t
t
r
(
o
b
j
,
 
f
u
n
c
.
_
_
n
a
m
e
_
_
,
 
f
u
n
c
)


 
 
 
 
r
e
t
u
r
n
 
f
u
n
c




d
e
f
 
l
o
g
g
e
d
(
l
e
v
e
l
,
 
n
a
m
e
=
N
o
n
e
,
 
m
e
s
s
a
g
e
=
N
o
n
e
)
:


 
 
 
 
'
'
'


 
 
 
 
A
d
d
 
l
o
g
g
i
n
g
 
t
o
 
a
 
f
u
n
c
t
i
o
n
.
 
l
e
v
e
l
 
i
s
 
t
h
e
 
l
o
g
g
i
n
g


 
 
 
 
l
e
v
e
l
,
 
n
a
m
e
 
i
s
 
t
h
e
 
l
o
g
g
e
r
 
n
a
m
e
,
 
a
n
d
 
m
e
s
s
a
g
e
 
i
s
 
t
h
e


 
 
 
 
l
o
g
 
m
e
s
s
a
g
e
.
 
I
f
 
n
a
m
e
 
a
n
d
 
m
e
s
s
a
g
e
 
a
r
e
n
'
t
 
s
p
e
c
i
f
i
e
d
,


 
 
 
 
t
h
e
y
 
d
e
f
a
u
l
t
 
t
o
 
t
h
e
 
f
u
n
c
t
i
o
n
'
s
 
m
o
d
u
l
e
 
a
n
d
 
n
a
m
e
.


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
d
e
c
o
r
a
t
e
(
f
u
n
c
)
:


 
 
 
 
 
 
 
 
l
o
g
n
a
m
e
 
=
 
n
a
m
e
 
i
f
 
n
a
m
e
 
e
l
s
e
 
f
u
n
c
.
_
_
m
o
d
u
l
e
_
_


 
 
 
 
 
 
 
 
l
o
g
 
=
 
l
o
g
g
i
n
g
.
g
e
t
L
o
g
g
e
r
(
l
o
g
n
a
m
e
)


 
 
 
 
 
 
 
 
l
o
g
m
s
g
 
=
 
m
e
s
s
a
g
e
 
i
f
 
m
e
s
s
a
g
e
 
e
l
s
e
 
f
u
n
c
.
_
_
n
a
m
e
_
_




 
 
 
 
 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
l
o
g
.
l
o
g
(
l
e
v
e
l
,
 
l
o
g
m
s
g
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
 
 
 
 
#
 
A
t
t
a
c
h
 
s
e
t
t
e
r
 
f
u
n
c
t
i
o
n
s


 
 
 
 
 
 
 
 
@
a
t
t
a
c
h
_
w
r
a
p
p
e
r
(
w
r
a
p
p
e
r
)


 
 
 
 
 
 
 
 
d
e
f
 
s
e
t
_
l
e
v
e
l
(
n
e
w
l
e
v
e
l
)
:


 
 
 
 
 
 
 
 
 
 
 
 
n
o
n
l
o
c
a
l
 
l
e
v
e
l


 
 
 
 
 
 
 
 
 
 
 
 
l
e
v
e
l
 
=
 
n
e
w
l
e
v
e
l




 
 
 
 
 
 
 
 
@
a
t
t
a
c
h
_
w
r
a
p
p
e
r
(
w
r
a
p
p
e
r
)


 
 
 
 
 
 
 
 
d
e
f
 
s
e
t
_
m
e
s
s
a
g
e
(
n
e
w
m
s
g
)
:


 
 
 
 
 
 
 
 
 
 
 
 
n
o
n
l
o
c
a
l
 
l
o
g
m
s
g


 
 
 
 
 
 
 
 
 
 
 
 
l
o
g
m
s
g
 
=
 
n
e
w
m
s
g




 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




 
 
 
 
r
e
t
u
r
n
 
d
e
c
o
r
a
t
e




#
 
E
x
a
m
p
l
e
 
u
s
e


@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
D
E
B
U
G
)


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y




@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
C
R
I
T
I
C
A
L
,
 
'
e
x
a
m
p
l
e
'
)


d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
!
'
)



下面是交互环境下的使用例子：

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

In [ ]:
# Change the log message
add.set_message('Add called')
add(2, 3)

In [ ]:
# Change the log level
add.set_level(logging.WARNING)
add(2, 3)

### 讨论


这一小节的关键点在于访问函数(如 set_message() 和 set_level() )，它们被作为属性赋给包装器。
每个访问函数允许使用 nonlocal 来修改函数内部的变量。

还有一个令人吃惊的地方是访问函数会在多层装饰器间传播(如果你的装饰器都使用了 @functools.wraps 注解)。
例如，假设你引入另外一个装饰器，比如9.2小节中的 @timethis ，像下面这样：

In [ ]:
@
t
i
m
e
t
h
i
s


@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
D
E
B
U
G
)


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
n
 
-
=
 
1



你会发现访问函数依旧有效：

In [ ]:
countdown(10000000)

In [ ]:
countdown.set_level(logging.WARNING)
countdown.set_message("Counting down to zero")
countdown(10000000)

你还会发现即使装饰器像下面这样以相反的方向排放，效果也是一样的：

In [ ]:
@
l
o
g
g
e
d
(
l
o
g
g
i
n
g
.
D
E
B
U
G
)


@
t
i
m
e
t
h
i
s


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
n
 
-
=
 
1



还能通过使用lambda表达式代码来让访问函数的返回不同的设定值：

In [ ]:
@
a
t
t
a
c
h
_
w
r
a
p
p
e
r
(
w
r
a
p
p
e
r
)


d
e
f
 
g
e
t
_
l
e
v
e
l
(
)
:


 
 
 
 
r
e
t
u
r
n
 
l
e
v
e
l




#
 
A
l
t
e
r
n
a
t
i
v
e


w
r
a
p
p
e
r
.
g
e
t
_
l
e
v
e
l
 
=
 
l
a
m
b
d
a
:
 
l
e
v
e
l



一个比较难理解的地方就是对于访问函数的首次使用。例如，你可能会考虑另外一个方法直接访问函数的属性，如下：

In [ ]:
@
w
r
a
p
s
(
f
u
n
c
)


d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
w
r
a
p
p
e
r
.
l
o
g
.
l
o
g
(
w
r
a
p
p
e
r
.
l
e
v
e
l
,
 
w
r
a
p
p
e
r
.
l
o
g
m
s
g
)


 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




#
 
A
t
t
a
c
h
 
a
d
j
u
s
t
a
b
l
e
 
a
t
t
r
i
b
u
t
e
s


w
r
a
p
p
e
r
.
l
e
v
e
l
 
=
 
l
e
v
e
l


w
r
a
p
p
e
r
.
l
o
g
m
s
g
 
=
 
l
o
g
m
s
g


w
r
a
p
p
e
r
.
l
o
g
 
=
 
l
o
g



这个方法也可能正常工作，但前提是它必须是最外层的装饰器才行。
如果它的上面还有另外的装饰器(比如上面提到的 @timethis 例子)，那么它会隐藏底层属性，使得修改它们没有任何作用。
而通过使用访问函数就能避免这样的局限性。

最后提一点，这一小节的方案也可以作为9.9小节中装饰器类的另一种实现方法。

## 9.6 带可选参数的装饰器


### 问题


你想写一个装饰器，既可以不传参数给它，比如 @decorator ，
也可以传递可选参数给它，比如 @decorator(x,y,z) 。

### 解决方案


下面是9.5小节中日志装饰器的一个修改版本：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s
,
 
p
a
r
t
i
a
l


i
m
p
o
r
t
 
l
o
g
g
i
n
g




d
e
f
 
l
o
g
g
e
d
(
f
u
n
c
=
N
o
n
e
,
 
*
,
 
l
e
v
e
l
=
l
o
g
g
i
n
g
.
D
E
B
U
G
,
 
n
a
m
e
=
N
o
n
e
,
 
m
e
s
s
a
g
e
=
N
o
n
e
)
:


 
 
 
 
i
f
 
f
u
n
c
 
i
s
 
N
o
n
e
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
p
a
r
t
i
a
l
(
l
o
g
g
e
d
,
 
l
e
v
e
l
=
l
e
v
e
l
,
 
n
a
m
e
=
n
a
m
e
,
 
m
e
s
s
a
g
e
=
m
e
s
s
a
g
e
)




 
 
 
 
l
o
g
n
a
m
e
 
=
 
n
a
m
e
 
i
f
 
n
a
m
e
 
e
l
s
e
 
f
u
n
c
.
_
_
m
o
d
u
l
e
_
_


 
 
 
 
l
o
g
 
=
 
l
o
g
g
i
n
g
.
g
e
t
L
o
g
g
e
r
(
l
o
g
n
a
m
e
)


 
 
 
 
l
o
g
m
s
g
 
=
 
m
e
s
s
a
g
e
 
i
f
 
m
e
s
s
a
g
e
 
e
l
s
e
 
f
u
n
c
.
_
_
n
a
m
e
_
_




 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
l
o
g
.
l
o
g
(
l
e
v
e
l
,
 
l
o
g
m
s
g
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




#
 
E
x
a
m
p
l
e
 
u
s
e


@
l
o
g
g
e
d


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y




@
l
o
g
g
e
d
(
l
e
v
e
l
=
l
o
g
g
i
n
g
.
C
R
I
T
I
C
A
L
,
 
n
a
m
e
=
'
e
x
a
m
p
l
e
'
)


d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
!
'
)



可以看到，@logged 装饰器可以同时不带参数或带参数。

### 讨论


这里提到的这个问题就是通常所说的编程一致性问题。
当我们使用装饰器的时候，大部分程序员习惯了要么不给它们传递任何参数，要么给它们传递确切参数。
其实从技术上来讲，我们可以定义一个所有参数都是可选的装饰器，就像下面这样：

In [ ]:
@
l
o
g
g
e
d
(
)


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
+
y



但是，这种写法并不符合我们的习惯，有时候程序员忘记加上后面的括号会导致错误。
这里我们向你展示了如何以一致的编程风格来同时满足没有括号和有括号两种情况。

为了理解代码是如何工作的，你需要非常熟悉装饰器是如何作用到函数上以及它们的调用规则。
对于一个像下面这样的简单装饰器：

In [ ]:
#
 
E
x
a
m
p
l
e
 
u
s
e


@
l
o
g
g
e
d


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y



这个调用序列跟下面等价：

In [ ]:
d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y




a
d
d
 
=
 
l
o
g
g
e
d
(
a
d
d
)



这时候，被装饰函数会被当做第一个参数直接传递给 logged 装饰器。
因此，logged() 中的第一个参数就是被包装函数本身。所有其他参数都必须有默认值。

而对于一个下面这样有参数的装饰器：

In [ ]:
@
l
o
g
g
e
d
(
l
e
v
e
l
=
l
o
g
g
i
n
g
.
C
R
I
T
I
C
A
L
,
 
n
a
m
e
=
'
e
x
a
m
p
l
e
'
)


d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
!
'
)



调用序列跟下面等价：

In [ ]:
d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
!
'
)


s
p
a
m
 
=
 
l
o
g
g
e
d
(
l
e
v
e
l
=
l
o
g
g
i
n
g
.
C
R
I
T
I
C
A
L
,
 
n
a
m
e
=
'
e
x
a
m
p
l
e
'
)
(
s
p
a
m
)



初始调用 logged() 函数时，被包装函数并没有传递进来。
因此在装饰器内，它必须是可选的。这个反过来会迫使其他参数必须使用关键字来指定。
并且，但这些参数被传递进来后，装饰器要返回一个接受一个函数参数并包装它的函数(参考9.5小节)。
为了这样做，我们使用了一个技巧，就是利用 functools.partial 。
它会返回一个未完全初始化的自身，除了被包装函数外其他参数都已经确定下来了。
可以参考7.8小节获取更多 partial() 方法的知识。

## 9.7 利用装饰器强制函数上的类型检查


### 问题


作为某种编程规约，你想在对函数参数进行强制类型检查。

### 解决方案


在演示实际代码前，先说明我们的目标：能对函数参数类型进行断言，类似下面这样：

In [ ]:
@typeassert(int, int)
def add(x, y):
    return x + y
add(2, 3)

In [ ]:
add(2, 'hello')

下面是使用装饰器技术来实现 @typeassert ：

In [ ]:
f
r
o
m
 
i
n
s
p
e
c
t
 
i
m
p
o
r
t
 
s
i
g
n
a
t
u
r
e


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




d
e
f
 
t
y
p
e
a
s
s
e
r
t
(
*
t
y
_
a
r
g
s
,
 
*
*
t
y
_
k
w
a
r
g
s
)
:


 
 
 
 
d
e
f
 
d
e
c
o
r
a
t
e
(
f
u
n
c
)
:


 
 
 
 
 
 
 
 
#
 
I
f
 
i
n
 
o
p
t
i
m
i
z
e
d
 
m
o
d
e
,
 
d
i
s
a
b
l
e
 
t
y
p
e
 
c
h
e
c
k
i
n
g


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
_
_
d
e
b
u
g
_
_
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c




 
 
 
 
 
 
 
 
#
 
M
a
p
 
f
u
n
c
t
i
o
n
 
a
r
g
u
m
e
n
t
 
n
a
m
e
s
 
t
o
 
s
u
p
p
l
i
e
d
 
t
y
p
e
s


 
 
 
 
 
 
 
 
s
i
g
 
=
 
s
i
g
n
a
t
u
r
e
(
f
u
n
c
)


 
 
 
 
 
 
 
 
b
o
u
n
d
_
t
y
p
e
s
 
=
 
s
i
g
.
b
i
n
d
_
p
a
r
t
i
a
l
(
*
t
y
_
a
r
g
s
,
 
*
*
t
y
_
k
w
a
r
g
s
)
.
a
r
g
u
m
e
n
t
s




 
 
 
 
 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
b
o
u
n
d
_
v
a
l
u
e
s
 
=
 
s
i
g
.
b
i
n
d
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
 
 
 
 
#
 
E
n
f
o
r
c
e
 
t
y
p
e
 
a
s
s
e
r
t
i
o
n
s
 
a
c
r
o
s
s
 
s
u
p
p
l
i
e
d
 
a
r
g
u
m
e
n
t
s


 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
v
a
l
u
e
 
i
n
 
b
o
u
n
d
_
v
a
l
u
e
s
.
a
r
g
u
m
e
n
t
s
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
a
m
e
 
i
n
 
b
o
u
n
d
_
t
y
p
e
s
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
b
o
u
n
d
_
t
y
p
e
s
[
n
a
m
e
]
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
A
r
g
u
m
e
n
t
 
{
}
 
m
u
s
t
 
b
e
 
{
}
'
.
f
o
r
m
a
t
(
n
a
m
e
,
 
b
o
u
n
d
_
t
y
p
e
s
[
n
a
m
e
]
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r


 
 
 
 
r
e
t
u
r
n
 
d
e
c
o
r
a
t
e



可以看出这个装饰器非常灵活，既可以指定所有参数类型，也可以只指定部分。
并且可以通过位置或关键字来指定参数类型。下面是使用示例：

In [ ]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)
spam(1, 2, 3)

In [ ]:
spam(1, 'hello', 3)

In [ ]:
spam(1, 'hello', 'world')

### 讨论


这节是高级装饰器示例，引入了很多重要的概念。

首先，装饰器只会在函数定义时被调用一次。
有时候你去掉装饰器的功能，那么你只需要简单的返回被装饰函数即可。
下面的代码中，如果全局变量　__debug__ 被设置成了False(当你使用-O或-OO参数的优化模式执行程序时)，
那么就直接返回未修改过的函数本身：

In [ ]:
d
e
f
 
d
e
c
o
r
a
t
e
(
f
u
n
c
)
:


 
 
 
 
#
 
I
f
 
i
n
 
o
p
t
i
m
i
z
e
d
 
m
o
d
e
,
 
d
i
s
a
b
l
e
 
t
y
p
e
 
c
h
e
c
k
i
n
g


 
 
 
 
i
f
 
n
o
t
 
_
_
d
e
b
u
g
_
_
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c



其次，这里还对被包装函数的参数签名进行了检查，我们使用了 inspect.signature() 函数。
简单来讲，它运行你提取一个可调用对象的参数签名信息。例如：

In [ ]:
from inspect import signature
def spam(x, y, z=42):
    pass
sig = signature(spam)
print(sig)

In [ ]:
sig.parameters

In [ ]:
sig.parameters['z'].name

In [ ]:
sig.parameters['z'].default

In [ ]:
sig.parameters['z'].kind

装饰器的开始部分，我们使用了 bind_partial() 方法来执行从指定类型到名称的部分绑定。
下面是例子演示：

In [ ]:
bound_types = sig.bind_partial(int,z=int)
bound_types

In [ ]:
bound_types.arguments

在这个部分绑定中，你可以注意到缺失的参数被忽略了(比如并没有对y进行绑定)。
不过最重要的是创建了一个有序字典 bound_types.arguments 。
这个字典会将参数名以函数签名中相同顺序映射到指定的类型值上面去。
在我们的装饰器例子中，这个映射包含了我们要强制指定的类型断言。

在装饰器创建的实际包装函数中使用到了 sig.bind() 方法。
bind() 跟 bind_partial() 类似，但是它不允许忽略任何参数。因此有了下面的结果：

In [ ]:
bound_values = sig.bind(1, 2, 3)
bound_values.arguments

使用这个映射我们可以很轻松的实现我们的强制类型检查：

In [ ]:
for name, value in bound_values.arguments.items():
    if name in bound_types.arguments:
        if not isinstance(value, bound_types.arguments[name]):
            raise TypeError()

不过这个方案还有点小瑕疵，它对于有默认值的参数并不适用。
比如下面的代码可以正常工作，尽管items的类型是错误的：

In [ ]:
@typeassert(int, list)
def bar(x, items=None):
    if items is None:
        items = []
    items.append(x)
    return items
bar(2)

In [ ]:
bar(2,3)

In [ ]:
bar(4, [1, 2, 3])

最后一点是关于适用装饰器参数和函数注解之间的争论。
例如，为什么不像下面这样写一个装饰器来查找函数中的注解呢？

In [ ]:
@
t
y
p
e
a
s
s
e
r
t


d
e
f
 
s
p
a
m
(
x
:
i
n
t
,
 
y
,
 
z
:
i
n
t
 
=
 
4
2
)
:


 
 
 
 
p
r
i
n
t
(
x
,
y
,
z
)



一个可能的原因是如果使用了函数参数注解，那么就被限制了。
如果注解被用来做类型检查就不能做其他事情了。而且 @typeassert 不能再用于使用注解做其他事情的函数了。
而使用上面的装饰器参数灵活性大多了，也更加通用。

可以在PEP 362以及 inspect 模块中找到更多关于函数参数对象的信息。在9.16小节还有另外一个例子。

## 9.8 将装饰器定义为类的一部分


### 问题


你想在类中定义装饰器，并将其作用在其他函数或方法上。

### 解决方案


在类里面定义装饰器很简单，但是你首先要确认它的使用方式。比如到底是作为一个实例方法还是类方法。
下面我们用例子来阐述它们的不同：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




c
l
a
s
s
 
A
:


 
 
 
 
#
 
D
e
c
o
r
a
t
o
r
 
a
s
 
a
n
 
i
n
s
t
a
n
c
e
 
m
e
t
h
o
d


 
 
 
 
d
e
f
 
d
e
c
o
r
a
t
o
r
1
(
s
e
l
f
,
 
f
u
n
c
)
:


 
 
 
 
 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
D
e
c
o
r
a
t
o
r
 
1
'
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




 
 
 
 
#
 
D
e
c
o
r
a
t
o
r
 
a
s
 
a
 
c
l
a
s
s
 
m
e
t
h
o
d


 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
d
e
c
o
r
a
t
o
r
2
(
c
l
s
,
 
f
u
n
c
)
:


 
 
 
 
 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
D
e
c
o
r
a
t
o
r
 
2
'
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r



下面是一使用例子：

In [ ]:
#
 
A
s
 
a
n
 
i
n
s
t
a
n
c
e
 
m
e
t
h
o
d


a
 
=
 
A
(
)


@
a
.
d
e
c
o
r
a
t
o
r
1


d
e
f
 
s
p
a
m
(
)
:


 
 
 
 
p
a
s
s


#
 
A
s
 
a
 
c
l
a
s
s
 
m
e
t
h
o
d


@
A
.
d
e
c
o
r
a
t
o
r
2


d
e
f
 
g
r
o
k
(
)
:


 
 
 
 
p
a
s
s



仔细观察可以发现一个是实例调用，一个是类调用。

### 讨论


在类中定义装饰器初看上去好像很奇怪，但是在标准库中有很多这样的例子。
特别的，@property 装饰器实际上是一个类，它里面定义了三个方法 getter(), setter(), deleter() ,
每一个方法都是一个装饰器。例如：

In [ ]:
c
l
a
s
s
 
P
e
r
s
o
n
:


 
 
 
 
#
 
C
r
e
a
t
e
 
a
 
p
r
o
p
e
r
t
y
 
i
n
s
t
a
n
c
e


 
 
 
 
f
i
r
s
t
_
n
a
m
e
 
=
 
p
r
o
p
e
r
t
y
(
)




 
 
 
 
#
 
A
p
p
l
y
 
d
e
c
o
r
a
t
o
r
 
m
e
t
h
o
d
s


 
 
 
 
@
f
i
r
s
t
_
n
a
m
e
.
g
e
t
t
e
r


 
 
 
 
d
e
f
 
f
i
r
s
t
_
n
a
m
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
f
i
r
s
t
_
n
a
m
e




 
 
 
 
@
f
i
r
s
t
_
n
a
m
e
.
s
e
t
t
e
r


 
 
 
 
d
e
f
 
f
i
r
s
t
_
n
a
m
e
(
s
e
l
f
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
s
t
r
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
E
x
p
e
c
t
e
d
 
a
 
s
t
r
i
n
g
'
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
f
i
r
s
t
_
n
a
m
e
 
=
 
v
a
l
u
e



它为什么要这么定义的主要原因是各种不同的装饰器方法会在关联的 property 实例上操作它的状态。
因此，任何时候只要你碰到需要在装饰器中记录或绑定信息，那么这不失为一种可行方法。

在类中定义装饰器有个难理解的地方就是对于额外参数 self 或 cls 的正确使用。
尽管最外层的装饰器函数比如 decorator1() 或 decorator2() 需要提供一个 self 或 cls 参数，
但是在两个装饰器内部被创建的 wrapper() 函数并不需要包含这个 self 参数。
你唯一需要这个参数是在你确实要访问包装器中这个实例的某些部分的时候。其他情况下都不用去管它。

对于类里面定义的包装器还有一点比较难理解，就是在涉及到继承的时候。
例如，假设你想让在A中定义的装饰器作用在子类B中。你需要像下面这样写：

In [ ]:
c
l
a
s
s
 
B
(
A
)
:


 
 
 
 
@
A
.
d
e
c
o
r
a
t
o
r
2


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
p
a
s
s



也就是说，装饰器要被定义成类方法并且你必须显式的使用父类名去调用它。
你不能使用 @B.decorator2 ，因为在方法定义时，这个类B还没有被创建。

## 9.9 将装饰器定义为类


### 问题


你想使用一个装饰器去包装函数，但是希望返回一个可调用的实例。
你需要让你的装饰器可以同时工作在类定义的内部和外部。

### 解决方案


为了将装饰器定义成一个实例，你需要确保它实现了 __call__() 和 __get__() 方法。
例如，下面的代码定义了一个类，它在其他函数上放置一个简单的记录层：

In [ ]:
i
m
p
o
r
t
 
t
y
p
e
s


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




c
l
a
s
s
 
P
r
o
f
i
l
e
d
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
f
u
n
c
)
:


 
 
 
 
 
 
 
 
w
r
a
p
s
(
f
u
n
c
)
(
s
e
l
f
)


 
 
 
 
 
 
 
 
s
e
l
f
.
n
c
a
l
l
s
 
=
 
0




 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
c
a
l
l
s
 
+
=
 
1


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
_
w
r
a
p
p
e
d
_
_
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
d
e
f
 
_
_
g
e
t
_
_
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
,
 
c
l
s
)
:


 
 
 
 
 
 
 
 
i
f
 
i
n
s
t
a
n
c
e
 
i
s
 
N
o
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
s
.
M
e
t
h
o
d
T
y
p
e
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
)



你可以将它当做一个普通的装饰器来使用，在类里面或外面都可以：

In [ ]:
@
P
r
o
f
i
l
e
d


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y




c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
@
P
r
o
f
i
l
e
d


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
x
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
s
e
l
f
,
 
x
)



在交互环境中的使用示例：

In [ ]:
add(2, 3)

In [ ]:
add(4, 5)

In [ ]:
add.ncalls

In [ ]:
s = Spam()
s.bar(1)

In [ ]:
s.bar(2)

In [ ]:
s.bar(3)

In [ ]:
Spam.bar.ncalls

### 讨论


将装饰器定义成类通常是很简单的。但是这里还是有一些细节需要解释下，特别是当你想将它作用在实例方法上的时候。

首先，使用 functools.wraps() 函数的作用跟之前还是一样，将被包装函数的元信息复制到可调用实例中去。

其次，通常很容易会忽视上面的 __get__() 方法。如果你忽略它，保持其他代码不变再次运行，
你会发现当你去调用被装饰实例方法时出现很奇怪的问题。例如：

In [ ]:
s = Spam()
s.bar(3)

出错原因是当方法函数在一个类中被查找时，它们的 __get__() 方法依据描述器协议被调用，
在8.9小节已经讲述过描述器协议了。在这里，__get__() 的目的是创建一个绑定方法对象
(最终会给这个方法传递self参数)。下面是一个例子来演示底层原理：

In [ ]:
s = Spam()
def grok(self, x):
    pass
grok.__get__(s, Spam)

__get__() 方法是为了确保绑定方法对象能被正确的创建。
type.MethodType() 手动创建一个绑定方法来使用。只有当实例被使用的时候绑定方法才会被创建。
如果这个方法是在类上面来访问，
那么 __get__() 中的instance参数会被设置成None并直接返回 Profiled 实例本身。
这样的话我们就可以提取它的 ncalls 属性了。

如果你想避免一些混乱，也可以考虑另外一个使用闭包和 nonlocal 变量实现的装饰器，这个在9.5小节有讲到。例如：

In [ ]:
i
m
p
o
r
t
 
t
y
p
e
s


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




d
e
f
 
p
r
o
f
i
l
e
d
(
f
u
n
c
)
:


 
 
 
 
n
c
a
l
l
s
 
=
 
0


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
n
o
n
l
o
c
a
l
 
n
c
a
l
l
s


 
 
 
 
 
 
 
 
n
c
a
l
l
s
 
+
=
 
1


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
w
r
a
p
p
e
r
.
n
c
a
l
l
s
 
=
 
l
a
m
b
d
a
:
 
n
c
a
l
l
s


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




#
 
E
x
a
m
p
l
e


@
p
r
o
f
i
l
e
d


d
e
f
 
a
d
d
(
x
,
 
y
)
:


 
 
 
 
r
e
t
u
r
n
 
x
 
+
 
y



这个方式跟之前的效果几乎一样，除了对于 ncalls 的访问现在是通过一个被绑定为属性的函数来实现，例如：

In [ ]:
add(2, 3)

In [ ]:
add(4, 5)

In [ ]:
add.ncalls()

## 9.10 为类和静态方法提供装饰器


### 问题


你想给类或静态方法提供装饰器。

### 解决方案


给类或静态方法提供装饰器是很简单的，不过要确保装饰器在 @classmethod 或 @staticmethod 之前。例如：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e


f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




#
 
A
 
s
i
m
p
l
e
 
d
e
c
o
r
a
t
o
r


d
e
f
 
t
i
m
e
t
h
i
s
(
f
u
n
c
)
:


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
t
a
r
t
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
r
 
=
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
e
n
d
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
e
n
d
-
s
t
a
r
t
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
r


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




#
 
C
l
a
s
s
 
i
l
l
u
s
t
r
a
t
i
n
g
 
a
p
p
l
i
c
a
t
i
o
n
 
o
f
 
t
h
e
 
d
e
c
o
r
a
t
o
r
 
t
o
 
d
i
f
f
e
r
e
n
t
 
k
i
n
d
s
 
o
f
 
m
e
t
h
o
d
s


c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
@
t
i
m
e
t
h
i
s


 
 
 
 
d
e
f
 
i
n
s
t
a
n
c
e
_
m
e
t
h
o
d
(
s
e
l
f
,
 
n
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
s
e
l
f
,
 
n
)


 
 
 
 
 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
n
 
-
=
 
1




 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
@
t
i
m
e
t
h
i
s


 
 
 
 
d
e
f
 
c
l
a
s
s
_
m
e
t
h
o
d
(
c
l
s
,
 
n
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
c
l
s
,
 
n
)


 
 
 
 
 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
n
 
-
=
 
1




 
 
 
 
@
s
t
a
t
i
c
m
e
t
h
o
d


 
 
 
 
@
t
i
m
e
t
h
i
s


 
 
 
 
d
e
f
 
s
t
a
t
i
c
_
m
e
t
h
o
d
(
n
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
n
)


 
 
 
 
 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
n
 
-
=
 
1



装饰后的类和静态方法可正常工作，只不过增加了额外的计时功能：

In [ ]:
s = Spam()
s.instance_method(1000000)

In [ ]:
Spam.class_method(1000000)

In [ ]:
Spam.static_method(1000000)

### 讨论


如果你把装饰器的顺序写错了就会出错。例如，假设你像下面这样写：

In [ ]:
c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
@
t
i
m
e
t
h
i
s


 
 
 
 
@
s
t
a
t
i
c
m
e
t
h
o
d


 
 
 
 
d
e
f
 
s
t
a
t
i
c
_
m
e
t
h
o
d
(
n
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
n
)


 
 
 
 
 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
 
 
 
 
n
 
-
=
 
1



那么你调用这个静态方法时就会报错：

In [ ]:
Spam.static_method(1000000)

问题在于 @classmethod 和 @staticmethod 实际上并不会创建可直接调用的对象，
而是创建特殊的描述器对象(参考8.9小节)。因此当你试着在其他装饰器中将它们当做函数来使用时就会出错。
确保这种装饰器出现在装饰器链中的第一个位置可以修复这个问题。

当我们在抽象基类中定义类方法和静态方法(参考8.12小节)时，这里讲到的知识就很有用了。
例如，如果你想定义一个抽象类方法，可以使用类似下面的代码：

In [ ]:
f
r
o
m
 
a
b
c
 
i
m
p
o
r
t
 
A
B
C
M
e
t
a
,
 
a
b
s
t
r
a
c
t
m
e
t
h
o
d


c
l
a
s
s
 
A
(
m
e
t
a
c
l
a
s
s
=
A
B
C
M
e
t
a
)
:


 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
@
a
b
s
t
r
a
c
t
m
e
t
h
o
d


 
 
 
 
d
e
f
 
m
e
t
h
o
d
(
c
l
s
)
:


 
 
 
 
 
 
 
 
p
a
s
s



在这段代码中，@classmethod 跟 @abstractmethod 两者的顺序是有讲究的，如果你调换它们的顺序就会出错。

## 9.11 装饰器为被包装函数增加参数


### 问题


你想在装饰器中给被包装函数增加额外的参数，但是不能影响这个函数现有的调用规则。

### 解决方案


可以使用关键字参数来给被包装函数增加额外参数。考虑下面的装饰器：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s




d
e
f
 
o
p
t
i
o
n
a
l
_
d
e
b
u
g
(
f
u
n
c
)
:


 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
'
,
 
f
u
n
c
.
_
_
n
a
m
e
_
_
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r



In [ ]:
@optional_debug
def spam(a,b,c):
    print(a,b,c)
spam(1,2,3)

In [ ]:
spam(1,2,3, debug=True)

### 讨论


通过装饰器来给被包装函数增加参数的做法并不常见。
尽管如此，有时候它可以避免一些重复代码。例如，如果你有下面这样的代码：

In [ ]:
d
e
f
 
a
(
x
,
 
d
e
b
u
g
=
F
a
l
s
e
)
:


 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
 
a
'
)




d
e
f
 
b
(
x
,
 
y
,
 
z
,
 
d
e
b
u
g
=
F
a
l
s
e
)
:


 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
 
b
'
)




d
e
f
 
c
(
x
,
 
y
,
 
d
e
b
u
g
=
F
a
l
s
e
)
:


 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
 
c
'
)



那么你可以将其重构成这样：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s


i
m
p
o
r
t
 
i
n
s
p
e
c
t




d
e
f
 
o
p
t
i
o
n
a
l
_
d
e
b
u
g
(
f
u
n
c
)
:


 
 
 
 
i
f
 
'
d
e
b
u
g
'
 
i
n
 
i
n
s
p
e
c
t
.
g
e
t
a
r
g
s
p
e
c
(
f
u
n
c
)
.
a
r
g
s
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
d
e
b
u
g
 
a
r
g
u
m
e
n
t
 
a
l
r
e
a
d
y
 
d
e
f
i
n
e
d
'
)




 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
'
,
 
f
u
n
c
.
_
_
n
a
m
e
_
_
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r




@
o
p
t
i
o
n
a
l
_
d
e
b
u
g


d
e
f
 
a
(
x
)
:


 
 
 
 
p
a
s
s




@
o
p
t
i
o
n
a
l
_
d
e
b
u
g


d
e
f
 
b
(
x
,
 
y
,
 
z
)
:


 
 
 
 
p
a
s
s




@
o
p
t
i
o
n
a
l
_
d
e
b
u
g


d
e
f
 
c
(
x
,
 
y
)
:


 
 
 
 
p
a
s
s



这种实现方案之所以行得通，在于强制关键字参数很容易被添加到接受 *args 和 **kwargs 参数的函数中。
通过使用强制关键字参数，它被作为一个特殊情况被挑选出来，
并且接下来仅仅使用剩余的位置和关键字参数去调用这个函数时，这个特殊参数会被排除在外。
也就是说，它并不会被纳入到 **kwargs 中去。

还有一个难点就是如何去处理被添加的参数与被包装函数参数直接的名字冲突。
例如，如果装饰器 @optional_debug 作用在一个已经拥有一个 debug 参数的函数上时会有问题。
这里我们增加了一步名字检查。

上面的方案还可以更完美一点，因为精明的程序员应该发现了被包装函数的函数签名其实是错误的。例如：

In [ ]:
@optional_debug
def add(x,y):
    return x+y
import inspect
print(inspect.signature(add))

通过如下的修改，可以解决这个问题：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
w
r
a
p
s


i
m
p
o
r
t
 
i
n
s
p
e
c
t




d
e
f
 
o
p
t
i
o
n
a
l
_
d
e
b
u
g
(
f
u
n
c
)
:


 
 
 
 
i
f
 
'
d
e
b
u
g
'
 
i
n
 
i
n
s
p
e
c
t
.
g
e
t
a
r
g
s
p
e
c
(
f
u
n
c
)
.
a
r
g
s
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
d
e
b
u
g
 
a
r
g
u
m
e
n
t
 
a
l
r
e
a
d
y
 
d
e
f
i
n
e
d
'
)




 
 
 
 
@
w
r
a
p
s
(
f
u
n
c
)


 
 
 
 
d
e
f
 
w
r
a
p
p
e
r
(
*
a
r
g
s
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
d
e
b
u
g
:


 
 
 
 
 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
a
l
l
i
n
g
'
,
 
f
u
n
c
.
_
_
n
a
m
e
_
_
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
s
i
g
 
=
 
i
n
s
p
e
c
t
.
s
i
g
n
a
t
u
r
e
(
f
u
n
c
)


 
 
 
 
p
a
r
m
s
 
=
 
l
i
s
t
(
s
i
g
.
p
a
r
a
m
e
t
e
r
s
.
v
a
l
u
e
s
(
)
)


 
 
 
 
p
a
r
m
s
.
a
p
p
e
n
d
(
i
n
s
p
e
c
t
.
P
a
r
a
m
e
t
e
r
(
'
d
e
b
u
g
'
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
n
s
p
e
c
t
.
P
a
r
a
m
e
t
e
r
.
K
E
Y
W
O
R
D
_
O
N
L
Y
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
d
e
f
a
u
l
t
=
F
a
l
s
e
)
)


 
 
 
 
w
r
a
p
p
e
r
.
_
_
s
i
g
n
a
t
u
r
e
_
_
 
=
 
s
i
g
.
r
e
p
l
a
c
e
(
p
a
r
a
m
e
t
e
r
s
=
p
a
r
m
s
)


 
 
 
 
r
e
t
u
r
n
 
w
r
a
p
p
e
r



通过这样的修改，包装后的函数签名就能正确的显示 debug 参数的存在了。例如：

In [ ]:
@optional_debug
def add(x,y):
    return x+y
print(inspect.signature(add))

In [ ]:
add(2,3)

参考9.16小节获取更多关于函数签名的信息。

## 9.12 使用装饰器扩充类的功能


### 问题


你想通过反省或者重写类定义的某部分来修改它的行为，但是你又不希望使用继承或元类的方式。

### 解决方案


这种情况可能是类装饰器最好的使用场景了。例如，下面是一个重写了特殊方法 __getattribute__ 的类装饰器，
可以打印日志：

In [ ]:
d
e
f
 
l
o
g
_
g
e
t
a
t
t
r
i
b
u
t
e
(
c
l
s
)
:


 
 
 
 
#
 
G
e
t
 
t
h
e
 
o
r
i
g
i
n
a
l
 
i
m
p
l
e
m
e
n
t
a
t
i
o
n


 
 
 
 
o
r
i
g
_
g
e
t
a
t
t
r
i
b
u
t
e
 
=
 
c
l
s
.
_
_
g
e
t
a
t
t
r
i
b
u
t
e
_
_




 
 
 
 
#
 
M
a
k
e
 
a
 
n
e
w
 
d
e
f
i
n
i
t
i
o
n


 
 
 
 
d
e
f
 
n
e
w
_
g
e
t
a
t
t
r
i
b
u
t
e
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
g
e
t
t
i
n
g
:
'
,
 
n
a
m
e
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
o
r
i
g
_
g
e
t
a
t
t
r
i
b
u
t
e
(
s
e
l
f
,
 
n
a
m
e
)




 
 
 
 
#
 
A
t
t
a
c
h
 
t
o
 
t
h
e
 
c
l
a
s
s
 
a
n
d
 
r
e
t
u
r
n


 
 
 
 
c
l
s
.
_
_
g
e
t
a
t
t
r
i
b
u
t
e
_
_
 
=
 
n
e
w
_
g
e
t
a
t
t
r
i
b
u
t
e


 
 
 
 
r
e
t
u
r
n
 
c
l
s




#
 
E
x
a
m
p
l
e
 
u
s
e


@
l
o
g
_
g
e
t
a
t
t
r
i
b
u
t
e


c
l
a
s
s
 
A
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
x
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
x
 
=
 
x


 
 
 
 
d
e
f
 
s
p
a
m
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
p
a
s
s



下面是使用效果：

In [ ]:
a = A(42)
a.x

In [ ]:
a.spam()

### 讨论


类装饰器通常可以作为其他高级技术比如混入或元类的一种非常简洁的替代方案。
比如，上面示例中的另外一种实现使用到继承：

In [ ]:
c
l
a
s
s
 
L
o
g
g
e
d
G
e
t
a
t
t
r
i
b
u
t
e
:


 
 
 
 
d
e
f
 
_
_
g
e
t
a
t
t
r
i
b
u
t
e
_
_
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
g
e
t
t
i
n
g
:
'
,
 
n
a
m
e
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
g
e
t
a
t
t
r
i
b
u
t
e
_
_
(
n
a
m
e
)




#
 
E
x
a
m
p
l
e
:


c
l
a
s
s
 
A
(
L
o
g
g
e
d
G
e
t
a
t
t
r
i
b
u
t
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
x
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
x
 
=
 
x


 
 
 
 
d
e
f
 
s
p
a
m
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
p
a
s
s



这种方案也行得通，但是为了去理解它，你就必须知道方法调用顺序、super() 以及其它8.7小节介绍的继承知识。
某种程度上来讲，类装饰器方案就显得更加直观，并且它不会引入新的继承体系。它的运行速度也更快一些，
因为他并不依赖 super() 函数。

如果你系想在一个类上面使用多个类装饰器，那么就需要注意下顺序问题。
例如，一个装饰器A会将其装饰的方法完整替换成另一种实现，
而另一个装饰器B只是简单的在其装饰的方法中添加点额外逻辑。
那么这时候装饰器A就需要放在装饰器B的前面。

你还可以回顾一下8.13小节另外一个关于类装饰器的有用的例子。

## 9.13 使用元类控制实例的创建


### 问题


你想通过改变实例创建方式来实现单例、缓存或其他类似的特性。

### 解决方案


Python程序员都知道，如果你定义了一个类，就能像函数一样的调用它来创建实例，例如：

In [ ]:
c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e




a
 
=
 
S
p
a
m
(
'
G
u
i
d
o
'
)


b
 
=
 
S
p
a
m
(
'
D
i
a
n
a
'
)



如果你想自定义这个步骤，你可以定义一个元类并自己实现 __call__() 方法。

为了演示，假设你不想任何人创建这个类的实例：

In [ ]:
c
l
a
s
s
 
N
o
I
n
s
t
a
n
c
e
s
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
"
C
a
n
'
t
 
i
n
s
t
a
n
t
i
a
t
e
 
d
i
r
e
c
t
l
y
"
)




#
 
E
x
a
m
p
l
e


c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
N
o
I
n
s
t
a
n
c
e
s
)
:


 
 
 
 
@
s
t
a
t
i
c
m
e
t
h
o
d


 
 
 
 
d
e
f
 
g
r
o
k
(
x
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
S
p
a
m
.
g
r
o
k
'
)



这样的话，用户只能调用这个类的静态方法，而不能使用通常的方法来创建它的实例。例如：

In [ ]:
Spam.grok(42)

In [ ]:
s = Spam()

现在，假如你想实现单例模式（只能创建唯一实例的类），实现起来也很简单：

In [ ]:
c
l
a
s
s
 
S
i
n
g
l
e
t
o
n
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
i
n
s
t
a
n
c
e
 
=
 
N
o
n
e


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)




 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
s
e
l
f
.
_
_
i
n
s
t
a
n
c
e
 
i
s
 
N
o
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
i
n
s
t
a
n
c
e
 
=
 
s
u
p
e
r
(
)
.
_
_
c
a
l
l
_
_
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
_
i
n
s
t
a
n
c
e


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
_
i
n
s
t
a
n
c
e




#
 
E
x
a
m
p
l
e


c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
S
i
n
g
l
e
t
o
n
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
r
e
a
t
i
n
g
 
S
p
a
m
'
)



那么Spam类就只能创建唯一的实例了，演示如下：

In [ ]:
a = Spam()

In [ ]:
b = Spam()
a is b

In [ ]:
c = Spam()
a is c

最后，假设你想创建8.25小节中那样的缓存实例。下面我们可以通过元类来实现：

In [ ]:
i
m
p
o
r
t
 
w
e
a
k
r
e
f




c
l
a
s
s
 
C
a
c
h
e
d
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
c
a
c
h
e
 
=
 
w
e
a
k
r
e
f
.
W
e
a
k
V
a
l
u
e
D
i
c
t
i
o
n
a
r
y
(
)




 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
a
r
g
s
 
i
n
 
s
e
l
f
.
_
_
c
a
c
h
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
_
c
a
c
h
e
[
a
r
g
s
]


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
o
b
j
 
=
 
s
u
p
e
r
(
)
.
_
_
c
a
l
l
_
_
(
*
a
r
g
s
)


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
c
a
c
h
e
[
a
r
g
s
]
 
=
 
o
b
j


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
o
b
j




#
 
E
x
a
m
p
l
e


c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
C
a
c
h
e
d
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
r
e
a
t
i
n
g
 
S
p
a
m
(
{
!
r
}
)
'
.
f
o
r
m
a
t
(
n
a
m
e
)
)


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e



然后我也来测试一下：

In [ ]:
a = Spam('Guido')

In [ ]:
b = Spam('Diana')

In [ ]:
c = Spam('Guido') # Cached
a is b

In [ ]:
a is c # Cached value returned

### 讨论


利用元类实现多种实例创建模式通常要比不使用元类的方式优雅得多。

假设你不使用元类，你可能需要将类隐藏在某些工厂函数后面。
比如为了实现一个单例，你你可能会像下面这样写：

In [ ]:
c
l
a
s
s
 
_
S
p
a
m
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
C
r
e
a
t
i
n
g
 
S
p
a
m
'
)




_
s
p
a
m
_
i
n
s
t
a
n
c
e
 
=
 
N
o
n
e




d
e
f
 
S
p
a
m
(
)
:


 
 
 
 
g
l
o
b
a
l
 
_
s
p
a
m
_
i
n
s
t
a
n
c
e




 
 
 
 
i
f
 
_
s
p
a
m
_
i
n
s
t
a
n
c
e
 
i
s
 
n
o
t
 
N
o
n
e
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
_
s
p
a
m
_
i
n
s
t
a
n
c
e


 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
_
s
p
a
m
_
i
n
s
t
a
n
c
e
 
=
 
_
S
p
a
m
(
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
_
s
p
a
m
_
i
n
s
t
a
n
c
e



尽管使用元类可能会涉及到比较高级点的技术，但是它的代码看起来会更加简洁舒服，而且也更加直观。

更多关于创建缓存实例、弱引用等内容，请参考8.25小节。

## 9.14 捕获类的属性定义顺序


### 问题


你想自动记录一个类中属性和方法定义的顺序，
然后可以利用它来做很多操作（比如序列化、映射到数据库等等）。

### 解决方案


利用元类可以很容易的捕获类的定义信息。下面是一个例子，使用了一个OrderedDict来记录描述器的定义顺序：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
O
r
d
e
r
e
d
D
i
c
t




#
 
A
 
s
e
t
 
o
f
 
d
e
s
c
r
i
p
t
o
r
s
 
f
o
r
 
v
a
r
i
o
u
s
 
t
y
p
e
s


c
l
a
s
s
 
T
y
p
e
d
:


 
 
 
 
_
e
x
p
e
c
t
e
d
_
t
y
p
e
 
=
 
t
y
p
e
(
N
o
n
e
)


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
=
N
o
n
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
a
m
e
 
=
 
n
a
m
e




 
 
 
 
d
e
f
 
_
_
s
e
t
_
_
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
s
e
l
f
.
_
e
x
p
e
c
t
e
d
_
t
y
p
e
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
E
x
p
e
c
t
e
d
 
'
 
+
 
s
t
r
(
s
e
l
f
.
_
e
x
p
e
c
t
e
d
_
t
y
p
e
)
)


 
 
 
 
 
 
 
 
i
n
s
t
a
n
c
e
.
_
_
d
i
c
t
_
_
[
s
e
l
f
.
_
n
a
m
e
]
 
=
 
v
a
l
u
e




c
l
a
s
s
 
I
n
t
e
g
e
r
(
T
y
p
e
d
)
:


 
 
 
 
_
e
x
p
e
c
t
e
d
_
t
y
p
e
 
=
 
i
n
t




c
l
a
s
s
 
F
l
o
a
t
(
T
y
p
e
d
)
:


 
 
 
 
_
e
x
p
e
c
t
e
d
_
t
y
p
e
 
=
 
f
l
o
a
t




c
l
a
s
s
 
S
t
r
i
n
g
(
T
y
p
e
d
)
:


 
 
 
 
_
e
x
p
e
c
t
e
d
_
t
y
p
e
 
=
 
s
t
r




#
 
M
e
t
a
c
l
a
s
s
 
t
h
a
t
 
u
s
e
s
 
a
n
 
O
r
d
e
r
e
d
D
i
c
t
 
f
o
r
 
c
l
a
s
s
 
b
o
d
y


c
l
a
s
s
 
O
r
d
e
r
e
d
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
d
 
=
 
d
i
c
t
(
c
l
s
d
i
c
t
)


 
 
 
 
 
 
 
 
o
r
d
e
r
 
=
 
[
]


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
v
a
l
u
e
 
i
n
 
c
l
s
d
i
c
t
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
T
y
p
e
d
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
.
_
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
o
r
d
e
r
.
a
p
p
e
n
d
(
n
a
m
e
)


 
 
 
 
 
 
 
 
d
[
'
_
o
r
d
e
r
'
]
 
=
 
o
r
d
e
r


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
d
)




 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
_
_
p
r
e
p
a
r
e
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
O
r
d
e
r
e
d
D
i
c
t
(
)



在这个元类中，执行类主体时描述器的定义顺序会被一个 OrderedDict``捕获到，
生成的有序名称从字典中提取出来并放入类属性 ``_order 中。这样的话类中的方法可以通过多种方式来使用它。
例如，下面是一个简单的类，使用这个排序字典来实现将一个类实例的数据序列化为一行CSV数据：

In [ ]:
c
l
a
s
s
 
S
t
r
u
c
t
u
r
e
(
m
e
t
a
c
l
a
s
s
=
O
r
d
e
r
e
d
M
e
t
a
)
:


 
 
 
 
d
e
f
 
a
s
_
c
s
v
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
'
,
'
.
j
o
i
n
(
s
t
r
(
g
e
t
a
t
t
r
(
s
e
l
f
,
n
a
m
e
)
)
 
f
o
r
 
n
a
m
e
 
i
n
 
s
e
l
f
.
_
o
r
d
e
r
)




#
 
E
x
a
m
p
l
e
 
u
s
e


c
l
a
s
s
 
S
t
o
c
k
(
S
t
r
u
c
t
u
r
e
)
:


 
 
 
 
n
a
m
e
 
=
 
S
t
r
i
n
g
(
)


 
 
 
 
s
h
a
r
e
s
 
=
 
I
n
t
e
g
e
r
(
)


 
 
 
 
p
r
i
c
e
 
=
 
F
l
o
a
t
(
)




 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
s
h
a
r
e
s
,
 
p
r
i
c
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
 
 
 
 
s
e
l
f
.
s
h
a
r
e
s
 
=
 
s
h
a
r
e
s


 
 
 
 
 
 
 
 
s
e
l
f
.
p
r
i
c
e
 
=
 
p
r
i
c
e



我们在交互式环境中测试一下这个Stock类：

In [ ]:
s = Stock('GOOG',100,490.1)
s.name

In [ ]:
s.as_csv()

In [ ]:
t = Stock('AAPL','a lot', 610.23)

### 讨论


本节一个关键点就是OrderedMeta元类中定义的 `` __prepare__()`` 方法。
这个方法会在开始定义类和它的父类的时候被执行。它必须返回一个映射对象以便在类定义体中被使用到。
我们这里通过返回了一个OrderedDict而不是一个普通的字典，可以很容易的捕获定义的顺序。

如果你想构造自己的类字典对象，可以很容易的扩展这个功能。比如，下面的这个修改方案可以防止重复的定义：

In [ ]:
f
r
o
m
 
c
o
l
l
e
c
t
i
o
n
s
 
i
m
p
o
r
t
 
O
r
d
e
r
e
d
D
i
c
t




c
l
a
s
s
 
N
o
D
u
p
O
r
d
e
r
e
d
D
i
c
t
(
O
r
d
e
r
e
d
D
i
c
t
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
c
l
s
n
a
m
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
c
l
s
n
a
m
e
 
=
 
c
l
s
n
a
m
e


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
)


 
 
 
 
d
e
f
 
_
_
s
e
t
i
t
e
m
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
a
m
e
 
i
n
 
s
e
l
f
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
{
}
 
a
l
r
e
a
d
y
 
d
e
f
i
n
e
d
 
i
n
 
{
}
'
.
f
o
r
m
a
t
(
n
a
m
e
,
 
s
e
l
f
.
c
l
s
n
a
m
e
)
)


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
s
e
t
i
t
e
m
_
_
(
n
a
m
e
,
 
v
a
l
u
e
)




c
l
a
s
s
 
O
r
d
e
r
e
d
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
d
 
=
 
d
i
c
t
(
c
l
s
d
i
c
t
)


 
 
 
 
 
 
 
 
d
[
'
_
o
r
d
e
r
'
]
 
=
 
[
n
a
m
e
 
f
o
r
 
n
a
m
e
 
i
n
 
c
l
s
d
i
c
t
 
i
f
 
n
a
m
e
[
0
]
 
!
=
 
'
_
'
]


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
d
)




 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
_
_
p
r
e
p
a
r
e
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
N
o
D
u
p
O
r
d
e
r
e
d
D
i
c
t
(
c
l
s
n
a
m
e
)



下面我们测试重复的定义会出现什么情况：

In [ ]:
class A(metaclass=OrderedMeta):
def spam(self):
pass
def spam(self):
pass

最后还有一点很重要，就是在 __new__() 方法中对于元类中被修改字典的处理。
尽管类使用了另外一个字典来定义，在构造最终的 class 对象的时候，
我们仍然需要将这个字典转换为一个正确的 dict 实例。
通过语句 d = dict(clsdict) 来完成这个效果。

对于很多应用程序而已，能够捕获类定义的顺序是一个看似不起眼却又非常重要的特性。
例如，在对象关系映射中，我们通常会看到下面这种方式定义的类：

In [ ]:
c
l
a
s
s
 
S
t
o
c
k
(
M
o
d
e
l
)
:


 
 
 
 
n
a
m
e
 
=
 
S
t
r
i
n
g
(
)


 
 
 
 
s
h
a
r
e
s
 
=
 
I
n
t
e
g
e
r
(
)


 
 
 
 
p
r
i
c
e
 
=
 
F
l
o
a
t
(
)



在框架底层，我们必须捕获定义的顺序来将对象映射到元组或数据库表中的行（就类似于上面例子中的 as_csv() 的功能）。
这节演示的技术非常简单，并且通常会比其他类似方法（通常都要在描述器类中维护一个隐藏的计数器）要简单的多。

## 9.15 定义有可选参数的元类


### 问题


你想定义一个元类，允许类定义时提供可选参数，这样可以控制或配置类型的创建过程。

### 解决方案


在定义类的时候，Python允许我们使用 ``metaclass``关键字参数来指定特定的元类。
例如使用抽象基类：

In [ ]:
f
r
o
m
 
a
b
c
 
i
m
p
o
r
t
 
A
B
C
M
e
t
a
,
 
a
b
s
t
r
a
c
t
m
e
t
h
o
d


c
l
a
s
s
 
I
S
t
r
e
a
m
(
m
e
t
a
c
l
a
s
s
=
A
B
C
M
e
t
a
)
:


 
 
 
 
@
a
b
s
t
r
a
c
t
m
e
t
h
o
d


 
 
 
 
d
e
f
 
r
e
a
d
(
s
e
l
f
,
 
m
a
x
s
i
z
e
=
N
o
n
e
)
:


 
 
 
 
 
 
 
 
p
a
s
s




 
 
 
 
@
a
b
s
t
r
a
c
t
m
e
t
h
o
d


 
 
 
 
d
e
f
 
w
r
i
t
e
(
s
e
l
f
,
 
d
a
t
a
)
:


 
 
 
 
 
 
 
 
p
a
s
s



然而，在自定义元类中我们还可以提供其他的关键字参数，如下所示：

In [ ]:
c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
M
y
M
e
t
a
,
 
d
e
b
u
g
=
T
r
u
e
,
 
s
y
n
c
h
r
o
n
i
z
e
=
T
r
u
e
)
:


 
 
 
 
p
a
s
s



为了使元类支持这些关键字参数，你必须确保在 __prepare__() , __new__() 和 __init__() 方法中
都使用强制关键字参数。就像下面这样：

In [ ]:
c
l
a
s
s
 
M
y
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
#
 
O
p
t
i
o
n
a
l


 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
_
_
p
r
e
p
a
r
e
_
_
(
c
l
s
,
 
n
a
m
e
,
 
b
a
s
e
s
,
 
*
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
s
y
n
c
h
r
o
n
i
z
e
=
F
a
l
s
e
)
:


 
 
 
 
 
 
 
 
#
 
C
u
s
t
o
m
 
p
r
o
c
e
s
s
i
n
g


 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
p
r
e
p
a
r
e
_
_
(
n
a
m
e
,
 
b
a
s
e
s
)




 
 
 
 
#
 
R
e
q
u
i
r
e
d


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
n
a
m
e
,
 
b
a
s
e
s
,
 
n
s
,
 
*
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
s
y
n
c
h
r
o
n
i
z
e
=
F
a
l
s
e
)
:


 
 
 
 
 
 
 
 
#
 
C
u
s
t
o
m
 
p
r
o
c
e
s
s
i
n
g


 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
n
e
w
_
_
(
c
l
s
,
 
n
a
m
e
,
 
b
a
s
e
s
,
 
n
s
)




 
 
 
 
#
 
R
e
q
u
i
r
e
d


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
b
a
s
e
s
,
 
n
s
,
 
*
,
 
d
e
b
u
g
=
F
a
l
s
e
,
 
s
y
n
c
h
r
o
n
i
z
e
=
F
a
l
s
e
)
:


 
 
 
 
 
 
 
 
#
 
C
u
s
t
o
m
 
p
r
o
c
e
s
s
i
n
g


 
 
 
 
 
 
 
 
p
a
s
s


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
n
a
m
e
,
 
b
a
s
e
s
,
 
n
s
)



### 讨论


给一个元类添加可选关键字参数需要你完全弄懂类创建的所有步骤，
因为这些参数会被传递给每一个相关的方法。
__prepare__() 方法在所有类定义开始执行前首先被调用，用来创建类命名空间。
通常来讲，这个方法只是简单的返回一个字典或其他映射对象。
__new__() 方法被用来实例化最终的类对象。它在类的主体被执行完后开始执行。
__init__() 方法最后被调用，用来执行其他的一些初始化工作。

当我们构造元类的时候，通常只需要定义一个 __new__() 或 __init__() 方法，但不是两个都定义。
但是，如果需要接受其他的关键字参数的话，这两个方法就要同时提供，并且都要提供对应的参数签名。
默认的 __prepare__() 方法接受任意的关键字参数，但是会忽略它们，
所以只有当这些额外的参数可能会影响到类命名空间的创建时你才需要去定义 __prepare__() 方法。

通过使用强制关键字参数，在类的创建过程中我们必须通过关键字来指定这些参数。

使用关键字参数配置一个元类还可以视作对类变量的一种替代方式。例如：

In [ ]:
c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
M
y
M
e
t
a
)
:


 
 
 
 
d
e
b
u
g
 
=
 
T
r
u
e


 
 
 
 
s
y
n
c
h
r
o
n
i
z
e
 
=
 
T
r
u
e


 
 
 
 
p
a
s
s



将这些属性定义为参数的好处在于它们不会污染类的名称空间，
这些属性仅仅只从属于类的创建阶段，而不是类中的语句执行阶段。
另外，它们在 __prepare__() 方法中是可以被访问的，因为这个方法会在所有类主体执行前被执行。
但是类变量只能在元类的 __new__() 和 __init__() 方法中可见。

## 9.16 *args和**kwargs的强制参数签名


### 问题


你有一个函数或方法，它使用*args和**kwargs作为参数，这样使得它比较通用，
但有时候你想检查传递进来的参数是不是某个你想要的类型。

### 解决方案


对任何涉及到操作函数调用签名的问题，你都应该使用 inspect 模块中的签名特性。
我们最主要关注两个类：Signature 和 Parameter 。下面是一个创建函数前面的交互例子：

In [ ]:
from inspect import Signature, Parameter
# Make a signature for a func(x, y=42, *, z=None)
parms = [ Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
        Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
        Parameter('z', Parameter.KEYWORD_ONLY, default=None) ]
sig = Signature(parms)
print(sig)

一旦你有了一个签名对象，你就可以使用它的 bind() 方法很容易的将它绑定到 *args 和 **kwargs 上去。
下面是一个简单的演示：

In [ ]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name,value)
# Try various examples
func(1, 2, z=3)

In [ ]:
func(1)

In [ ]:
func(1, z=3)

In [ ]:
func(y=2, x=1)

In [ ]:
func(1, 2, 3, 4)

In [ ]:
func(y=2)

In [ ]:
func(1, y=2, x=3)

可以看出来，通过将签名和传递的参数绑定起来，可以强制函数调用遵循特定的规则，比如必填、默认、重复等等。

下面是一个强制函数签名更具体的例子。在代码中，我们在基类中先定义了一个非常通用的 __init__() 方法，
然后我们强制所有的子类必须提供一个特定的参数签名。

In [ ]:
f
r
o
m
 
i
n
s
p
e
c
t
 
i
m
p
o
r
t
 
S
i
g
n
a
t
u
r
e
,
 
P
a
r
a
m
e
t
e
r




d
e
f
 
m
a
k
e
_
s
i
g
(
*
n
a
m
e
s
)
:


 
 
 
 
p
a
r
m
s
 
=
 
[
P
a
r
a
m
e
t
e
r
(
n
a
m
e
,
 
P
a
r
a
m
e
t
e
r
.
P
O
S
I
T
I
O
N
A
L
_
O
R
_
K
E
Y
W
O
R
D
)


 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
 
i
n
 
n
a
m
e
s
]


 
 
 
 
r
e
t
u
r
n
 
S
i
g
n
a
t
u
r
e
(
p
a
r
m
s
)




c
l
a
s
s
 
S
t
r
u
c
t
u
r
e
:


 
 
 
 
_
_
s
i
g
n
a
t
u
r
e
_
_
 
=
 
m
a
k
e
_
s
i
g
(
)


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
b
o
u
n
d
_
v
a
l
u
e
s
 
=
 
s
e
l
f
.
_
_
s
i
g
n
a
t
u
r
e
_
_
.
b
i
n
d
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
v
a
l
u
e
 
i
n
 
b
o
u
n
d
_
v
a
l
u
e
s
.
a
r
g
u
m
e
n
t
s
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
t
a
t
t
r
(
s
e
l
f
,
 
n
a
m
e
,
 
v
a
l
u
e
)




#
 
E
x
a
m
p
l
e
 
u
s
e


c
l
a
s
s
 
S
t
o
c
k
(
S
t
r
u
c
t
u
r
e
)
:


 
 
 
 
_
_
s
i
g
n
a
t
u
r
e
_
_
 
=
 
m
a
k
e
_
s
i
g
(
'
n
a
m
e
'
,
 
'
s
h
a
r
e
s
'
,
 
'
p
r
i
c
e
'
)




c
l
a
s
s
 
P
o
i
n
t
(
S
t
r
u
c
t
u
r
e
)
:


 
 
 
 
_
_
s
i
g
n
a
t
u
r
e
_
_
 
=
 
m
a
k
e
_
s
i
g
(
'
x
'
,
 
'
y
'
)



下面是使用这个 Stock 类的示例：

In [ ]:
import inspect
print(inspect.signature(Stock))

In [ ]:
s1 = Stock('ACME', 100, 490.1)
s2 = Stock('ACME', 100)

In [ ]:
s3 = Stock('ACME', 100, 490.1, shares=50)

### 讨论


在我们需要构建通用函数库、编写装饰器或实现代理的时候，对于 *args 和 **kwargs 的使用是很普遍的。
但是，这样的函数有一个缺点就是当你想要实现自己的参数检验时，代码就会笨拙混乱。在8.11小节里面有这样一个例子。
这时候我们可以通过一个签名对象来简化它。

在最后的一个方案实例中，我们还可以通过使用自定义元类来创建签名对象。下面演示怎样来实现：

In [ ]:
f
r
o
m
 
i
n
s
p
e
c
t
 
i
m
p
o
r
t
 
S
i
g
n
a
t
u
r
e
,
 
P
a
r
a
m
e
t
e
r




d
e
f
 
m
a
k
e
_
s
i
g
(
*
n
a
m
e
s
)
:


 
 
 
 
p
a
r
m
s
 
=
 
[
P
a
r
a
m
e
t
e
r
(
n
a
m
e
,
 
P
a
r
a
m
e
t
e
r
.
P
O
S
I
T
I
O
N
A
L
_
O
R
_
K
E
Y
W
O
R
D
)


 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
 
i
n
 
n
a
m
e
s
]


 
 
 
 
r
e
t
u
r
n
 
S
i
g
n
a
t
u
r
e
(
p
a
r
m
s
)




c
l
a
s
s
 
S
t
r
u
c
t
u
r
e
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
c
l
s
d
i
c
t
[
'
_
_
s
i
g
n
a
t
u
r
e
_
_
'
]
 
=
 
m
a
k
e
_
s
i
g
(
*
c
l
s
d
i
c
t
.
g
e
t
(
'
_
f
i
e
l
d
s
'
,
[
]
)
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)




c
l
a
s
s
 
S
t
r
u
c
t
u
r
e
(
m
e
t
a
c
l
a
s
s
=
S
t
r
u
c
t
u
r
e
M
e
t
a
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
]


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
b
o
u
n
d
_
v
a
l
u
e
s
 
=
 
s
e
l
f
.
_
_
s
i
g
n
a
t
u
r
e
_
_
.
b
i
n
d
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
v
a
l
u
e
 
i
n
 
b
o
u
n
d
_
v
a
l
u
e
s
.
a
r
g
u
m
e
n
t
s
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
t
a
t
t
r
(
s
e
l
f
,
 
n
a
m
e
,
 
v
a
l
u
e
)




#
 
E
x
a
m
p
l
e


c
l
a
s
s
 
S
t
o
c
k
(
S
t
r
u
c
t
u
r
e
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
'
n
a
m
e
'
,
 
'
s
h
a
r
e
s
'
,
 
'
p
r
i
c
e
'
]




c
l
a
s
s
 
P
o
i
n
t
(
S
t
r
u
c
t
u
r
e
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
'
x
'
,
 
'
y
'
]



当我们自定义签名的时候，将签名存储在特定的属性 __signature__ 中通常是很有用的。
这样的话，在使用 inspect 模块执行内省的代码就能发现签名并将它作为调用约定。

In [ ]:
import inspect
print(inspect.signature(Stock))

In [ ]:
print(inspect.signature(Point))

## 9.17 在类上强制使用编程规约


### 问题


你的程序包含一个很大的类继承体系，你希望强制执行某些编程规约（或者代码诊断）来帮助程序员保持清醒。

### 解决方案


如果你想监控类的定义，通常可以通过定义一个元类。一个基本元类通常是继承自 type 并重定义它的 __new__() 方法
或者是 __init__() 方法。比如：

In [ ]:
c
l
a
s
s
 
M
y
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
s
e
l
f
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
#
 
c
l
s
n
a
m
e
 
i
s
 
n
a
m
e
 
o
f
 
c
l
a
s
s
 
b
e
i
n
g
 
d
e
f
i
n
e
d


 
 
 
 
 
 
 
 
#
 
b
a
s
e
s
 
i
s
 
t
u
p
l
e
 
o
f
 
b
a
s
e
 
c
l
a
s
s
e
s


 
 
 
 
 
 
 
 
#
 
c
l
s
d
i
c
t
 
i
s
 
c
l
a
s
s
 
d
i
c
t
i
o
n
a
r
y


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)



另一种是，定义 __init__() 方法：

In [ ]:
c
l
a
s
s
 
M
y
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)


 
 
 
 
 
 
 
 
#
 
c
l
s
n
a
m
e
 
i
s
 
n
a
m
e
 
o
f
 
c
l
a
s
s
 
b
e
i
n
g
 
d
e
f
i
n
e
d


 
 
 
 
 
 
 
 
#
 
b
a
s
e
s
 
i
s
 
t
u
p
l
e
 
o
f
 
b
a
s
e
 
c
l
a
s
s
e
s


 
 
 
 
 
 
 
 
#
 
c
l
s
d
i
c
t
 
i
s
 
c
l
a
s
s
 
d
i
c
t
i
o
n
a
r
y



为了使用这个元类，你通常要将它放到到一个顶级父类定义中，然后其他的类继承这个顶级父类。例如：

In [ ]:
c
l
a
s
s
 
R
o
o
t
(
m
e
t
a
c
l
a
s
s
=
M
y
M
e
t
a
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
A
(
R
o
o
t
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
B
(
R
o
o
t
)
:


 
 
 
 
p
a
s
s



元类的一个关键特点是它允许你在定义的时候检查类的内容。在重新定义 __init__() 方法中，
你可以很轻松的检查类字典、父类等等。并且，一旦某个元类被指定给了某个类，那么就会被继承到所有子类中去。
因此，一个框架的构建者就能在大型的继承体系中通过给一个顶级父类指定一个元类去捕获所有下面子类的定义。

作为一个具体的应用例子，下面定义了一个元类，它会拒绝任何有混合大小写名字作为方法的类定义（可能是想气死Java程序员^_^）：

In [ ]:
c
l
a
s
s
 
N
o
M
i
x
e
d
C
a
s
e
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
 
i
n
 
c
l
s
d
i
c
t
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
a
m
e
.
l
o
w
e
r
(
)
 
!
=
 
n
a
m
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
B
a
d
 
a
t
t
r
i
b
u
t
e
 
n
a
m
e
:
 
'
 
+
 
n
a
m
e
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)




c
l
a
s
s
 
R
o
o
t
(
m
e
t
a
c
l
a
s
s
=
N
o
M
i
x
e
d
C
a
s
e
M
e
t
a
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
A
(
R
o
o
t
)
:


 
 
 
 
d
e
f
 
f
o
o
_
b
a
r
(
s
e
l
f
)
:
 
#
 
O
k


 
 
 
 
 
 
 
 
p
a
s
s




c
l
a
s
s
 
B
(
R
o
o
t
)
:


 
 
 
 
d
e
f
 
f
o
o
B
a
r
(
s
e
l
f
)
:
 
#
 
T
y
p
e
E
r
r
o
r


 
 
 
 
 
 
 
 
p
a
s
s



作为更高级和实用的例子，下面有一个元类，它用来检测重载方法，确保它的调用参数跟父类中原始方法有着相同的参数签名。

In [ ]:
f
r
o
m
 
i
n
s
p
e
c
t
 
i
m
p
o
r
t
 
s
i
g
n
a
t
u
r
e


i
m
p
o
r
t
 
l
o
g
g
i
n
g




c
l
a
s
s
 
M
a
t
c
h
S
i
g
n
a
t
u
r
e
s
M
e
t
a
(
t
y
p
e
)
:




 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)


 
 
 
 
 
 
 
 
s
u
p
 
=
 
s
u
p
e
r
(
s
e
l
f
,
 
s
e
l
f
)


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
v
a
l
u
e
 
i
n
 
c
l
s
d
i
c
t
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
a
m
e
.
s
t
a
r
t
s
w
i
t
h
(
'
_
'
)
 
o
r
 
n
o
t
 
c
a
l
l
a
b
l
e
(
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
 
 
 
 
#
 
G
e
t
 
t
h
e
 
p
r
e
v
i
o
u
s
 
d
e
f
i
n
i
t
i
o
n
 
(
i
f
 
a
n
y
)
 
a
n
d
 
c
o
m
p
a
r
e
 
t
h
e
 
s
i
g
n
a
t
u
r
e
s


 
 
 
 
 
 
 
 
 
 
 
 
p
r
e
v
_
d
f
n
 
=
 
g
e
t
a
t
t
r
(
s
u
p
,
n
a
m
e
,
N
o
n
e
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
r
e
v
_
d
f
n
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
p
r
e
v
_
s
i
g
 
=
 
s
i
g
n
a
t
u
r
e
(
p
r
e
v
_
d
f
n
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
_
s
i
g
 
=
 
s
i
g
n
a
t
u
r
e
(
v
a
l
u
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
r
e
v
_
s
i
g
 
!
=
 
v
a
l
_
s
i
g
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
o
g
g
i
n
g
.
w
a
r
n
i
n
g
(
'
S
i
g
n
a
t
u
r
e
 
m
i
s
m
a
t
c
h
 
i
n
 
%
s
.
 
%
s
 
!
=
 
%
s
'
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
v
a
l
u
e
.
_
_
q
u
a
l
n
a
m
e
_
_
,
 
p
r
e
v
_
s
i
g
,
 
v
a
l
_
s
i
g
)




#
 
E
x
a
m
p
l
e


c
l
a
s
s
 
R
o
o
t
(
m
e
t
a
c
l
a
s
s
=
M
a
t
c
h
S
i
g
n
a
t
u
r
e
s
M
e
t
a
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
A
(
R
o
o
t
)
:


 
 
 
 
d
e
f
 
f
o
o
(
s
e
l
f
,
 
x
,
 
y
)
:


 
 
 
 
 
 
 
 
p
a
s
s




 
 
 
 
d
e
f
 
s
p
a
m
(
s
e
l
f
,
 
x
,
 
*
,
 
z
)
:


 
 
 
 
 
 
 
 
p
a
s
s




#
 
C
l
a
s
s
 
w
i
t
h
 
r
e
d
e
f
i
n
e
d
 
m
e
t
h
o
d
s
,
 
b
u
t
 
s
l
i
g
h
t
l
y
 
d
i
f
f
e
r
e
n
t
 
s
i
g
n
a
t
u
r
e
s


c
l
a
s
s
 
B
(
A
)
:


 
 
 
 
d
e
f
 
f
o
o
(
s
e
l
f
,
 
a
,
 
b
)
:


 
 
 
 
 
 
 
 
p
a
s
s




 
 
 
 
d
e
f
 
s
p
a
m
(
s
e
l
f
,
x
,
z
)
:


 
 
 
 
 
 
 
 
p
a
s
s



如果你运行这段代码，就会得到下面这样的输出结果：

In [ ]:
W
A
R
N
I
N
G
:
r
o
o
t
:
S
i
g
n
a
t
u
r
e
 
m
i
s
m
a
t
c
h
 
i
n
 
B
.
s
p
a
m
.
 
(
s
e
l
f
,
 
x
,
 
*
,
 
z
)
 
!
=
 
(
s
e
l
f
,
 
x
,
 
z
)


W
A
R
N
I
N
G
:
r
o
o
t
:
S
i
g
n
a
t
u
r
e
 
m
i
s
m
a
t
c
h
 
i
n
 
B
.
f
o
o
.
 
(
s
e
l
f
,
 
x
,
 
y
)
 
!
=
 
(
s
e
l
f
,
 
a
,
 
b
)



这种警告信息对于捕获一些微妙的程序bug是很有用的。例如，如果某个代码依赖于传递给方法的关键字参数，
那么当子类改变参数名字的时候就会调用出错。

### 讨论


在大型面向对象的程序中，通常将类的定义放在元类中控制是很有用的。
元类可以监控类的定义，警告编程人员某些没有注意到的可能出现的问题。

有人可能会说，像这样的错误可以通过程序分析工具或IDE去做会更好些。诚然，这些工具是很有用。
但是，如果你在构建一个框架或函数库供其他人使用，那么你没办法去控制使用者要使用什么工具。
因此，对于这种类型的程序，如果可以在元类中做检测或许可以带来更好的用户体验。

在元类中选择重新定义 __new__() 方法还是 __init__() 方法取决于你想怎样使用结果类。
__new__() 方法在类创建之前被调用，通常用于通过某种方式（比如通过改变类字典的内容）修改类的定义。
而 __init__() 方法是在类被创建之后被调用，当你需要完整构建类对象的时候会很有用。
在最后一个例子中，这是必要的，因为它使用了 super() 函数来搜索之前的定义。
它只能在类的实例被创建之后，并且相应的方法解析顺序也已经被设置好了。

最后一个例子还演示了Python的函数签名对象的使用。
实际上，元类将每个可调用定义放在一个类中，搜索前一个定义（如果有的话），
然后通过使用 inspect.signature() 来简单的比较它们的调用签名。

最后一点，代码中有一行使用了 super(self, self) 并不是排版错误。
当使用元类的时候，我们要时刻记住一点就是 self 实际上是一个类对象。
因此，这条语句其实就是用来寻找位于继承体系中构建 self 父类的定义。

## 9.18 以编程方式定义类


### 问题


你在写一段代码，最终需要创建一个新的类对象。你考虑将类的定义源代码以字符串的形式发布出去。
并且使用函数比如 exec() 来执行它，但是你想寻找一个更加优雅的解决方案。

### 解决方案


你可以使用函数 types.new_class() 来初始化新的类对象。
你需要做的只是提供类的名字、父类元组、关键字参数，以及一个用成员变量填充类字典的回调函数。例如：

In [ ]:
#
 
s
t
o
c
k
.
p
y


#
 
E
x
a
m
p
l
e
 
o
f
 
m
a
k
i
n
g
 
a
 
c
l
a
s
s
 
m
a
n
u
a
l
l
y
 
f
r
o
m
 
p
a
r
t
s




#
 
M
e
t
h
o
d
s


d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
s
h
a
r
e
s
,
 
p
r
i
c
e
)
:


 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
s
e
l
f
.
s
h
a
r
e
s
 
=
 
s
h
a
r
e
s


 
 
 
 
s
e
l
f
.
p
r
i
c
e
 
=
 
p
r
i
c
e


d
e
f
 
c
o
s
t
(
s
e
l
f
)
:


 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
s
h
a
r
e
s
 
*
 
s
e
l
f
.
p
r
i
c
e




c
l
s
_
d
i
c
t
 
=
 
{


 
 
 
 
'
_
_
i
n
i
t
_
_
'
 
:
 
_
_
i
n
i
t
_
_
,


 
 
 
 
'
c
o
s
t
'
 
:
 
c
o
s
t
,


}




#
 
M
a
k
e
 
a
 
c
l
a
s
s


i
m
p
o
r
t
 
t
y
p
e
s




S
t
o
c
k
 
=
 
t
y
p
e
s
.
n
e
w
_
c
l
a
s
s
(
'
S
t
o
c
k
'
,
 
(
)
,
 
{
}
,
 
l
a
m
b
d
a
 
n
s
:
 
n
s
.
u
p
d
a
t
e
(
c
l
s
_
d
i
c
t
)
)


S
t
o
c
k
.
_
_
m
o
d
u
l
e
_
_
 
=
 
_
_
n
a
m
e
_
_



这种方式会构建一个普通的类对象，并且按照你的期望工作：

In [ ]:
s = Stock('ACME', 50, 91.1)
s

In [ ]:
s.cost()

这种方法中，一个比较难理解的地方是在调用完 types.new_class() 对 Stock.__module__ 的赋值。
每次当一个类被定义后，它的 __module__ 属性包含定义它的模块名。
这个名字用于生成 __repr__() 方法的输出。它同样也被用于很多库，比如 pickle 。
因此，为了让你创建的类是“正确”的，你需要确保这个属性也设置正确了。

如果你想创建的类需要一个不同的元类，可以通过 types.new_class() 第三个参数传递给它。例如：

In [ ]:
import abc
Stock = types.new_class('Stock', (), {'metaclass': abc.ABCMeta},
                        lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__
Stock

In [ ]:
type(Stock)

第三个参数还可以包含其他的关键字参数。比如，一个类的定义如下：

In [ ]:
c
l
a
s
s
 
S
p
a
m
(
B
a
s
e
,
 
d
e
b
u
g
=
T
r
u
e
,
 
t
y
p
e
c
h
e
c
k
=
F
a
l
s
e
)
:


 
 
 
 
p
a
s
s



那么可以将其翻译成如下的 new_class() 调用形式：

In [ ]:
S
p
a
m
 
=
 
t
y
p
e
s
.
n
e
w
_
c
l
a
s
s
(
'
S
p
a
m
'
,
 
(
B
a
s
e
,
)
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
{
'
d
e
b
u
g
'
:
 
T
r
u
e
,
 
'
t
y
p
e
c
h
e
c
k
'
:
 
F
a
l
s
e
}
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
m
b
d
a
 
n
s
:
 
n
s
.
u
p
d
a
t
e
(
c
l
s
_
d
i
c
t
)
)



new_class() 第四个参数最神秘，它是一个用来接受类命名空间的映射对象的函数。
通常这是一个普通的字典，但是它实际上是 __prepare__() 方法返回的任意对象，这个在9.14小节已经介绍过了。
这个函数需要使用上面演示的 update() 方法给命名空间增加内容。

### 讨论


很多时候如果能构造新的类对象是很有用的。
有个很熟悉的例子是调用 collections.namedtuple() 函数，例如：

In [ ]:
Stock = collections.namedtuple('Stock', ['name', 'shares', 'price'])
Stock

namedtuple() 使用 exec() 而不是上面介绍的技术。但是，下面通过一个简单的变化，
我们直接创建一个类：

In [ ]:
i
m
p
o
r
t
 
o
p
e
r
a
t
o
r


i
m
p
o
r
t
 
t
y
p
e
s


i
m
p
o
r
t
 
s
y
s




d
e
f
 
n
a
m
e
d
_
t
u
p
l
e
(
c
l
a
s
s
n
a
m
e
,
 
f
i
e
l
d
n
a
m
e
s
)
:


 
 
 
 
#
 
P
o
p
u
l
a
t
e
 
a
 
d
i
c
t
i
o
n
a
r
y
 
o
f
 
f
i
e
l
d
 
p
r
o
p
e
r
t
y
 
a
c
c
e
s
s
o
r
s


 
 
 
 
c
l
s
_
d
i
c
t
 
=
 
{
 
n
a
m
e
:
 
p
r
o
p
e
r
t
y
(
o
p
e
r
a
t
o
r
.
i
t
e
m
g
e
t
t
e
r
(
n
)
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
,
 
n
a
m
e
 
i
n
 
e
n
u
m
e
r
a
t
e
(
f
i
e
l
d
n
a
m
e
s
)
 
}




 
 
 
 
#
 
M
a
k
e
 
a
 
_
_
n
e
w
_
_
 
f
u
n
c
t
i
o
n
 
a
n
d
 
a
d
d
 
t
o
 
t
h
e
 
c
l
a
s
s
 
d
i
c
t


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
a
r
g
s
)
 
!
=
 
l
e
n
(
f
i
e
l
d
n
a
m
e
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
E
x
p
e
c
t
e
d
 
{
}
 
a
r
g
u
m
e
n
t
s
'
.
f
o
r
m
a
t
(
l
e
n
(
f
i
e
l
d
n
a
m
e
s
)
)
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
u
p
l
e
.
_
_
n
e
w
_
_
(
c
l
s
,
 
a
r
g
s
)




 
 
 
 
c
l
s
_
d
i
c
t
[
'
_
_
n
e
w
_
_
'
]
 
=
 
_
_
n
e
w
_
_




 
 
 
 
#
 
M
a
k
e
 
t
h
e
 
c
l
a
s
s


 
 
 
 
c
l
s
 
=
 
t
y
p
e
s
.
n
e
w
_
c
l
a
s
s
(
c
l
a
s
s
n
a
m
e
,
 
(
t
u
p
l
e
,
)
,
 
{
}
,


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
l
a
m
b
d
a
 
n
s
:
 
n
s
.
u
p
d
a
t
e
(
c
l
s
_
d
i
c
t
)
)




 
 
 
 
#
 
S
e
t
 
t
h
e
 
m
o
d
u
l
e
 
t
o
 
t
h
a
t
 
o
f
 
t
h
e
 
c
a
l
l
e
r


 
 
 
 
c
l
s
.
_
_
m
o
d
u
l
e
_
_
 
=
 
s
y
s
.
_
g
e
t
f
r
a
m
e
(
1
)
.
f
_
g
l
o
b
a
l
s
[
'
_
_
n
a
m
e
_
_
'
]


 
 
 
 
r
e
t
u
r
n
 
c
l
s



这段代码的最后部分使用了一个所谓的”框架魔法”，通过调用 sys._getframe() 来获取调用者的模块名。
另外一个框架魔法例子在2.15小节中有介绍过。

下面的例子演示了前面的代码是如何工作的：

In [ ]:
Point = named_tuple('Point', ['x', 'y'])
Point

In [ ]:
p = Point(4, 5)
len(p)

In [ ]:
p.x

In [ ]:
p.y

In [ ]:
p.x = 2

In [ ]:
print('%s %s' % p)

这项技术一个很重要的方面是它对于元类的正确使用。
你可能像通过直接实例化一个元类来直接创建一个类：

In [ ]:
S
t
o
c
k
 
=
 
t
y
p
e
(
'
S
t
o
c
k
'
,
 
(
)
,
 
c
l
s
_
d
i
c
t
)



这种方法的问题在于它忽略了一些关键步骤，比如对于元类中 __prepare__() 方法的调用。
通过使用 types.new_class() ，你可以保证所有的必要初始化步骤都能得到执行。
比如，types.new_class() 第四个参数的回调函数接受 __prepare__() 方法返回的映射对象。

如果你仅仅只是想执行准备步骤，可以使用 types.prepare_class() 。例如：

In [ ]:
i
m
p
o
r
t
 
t
y
p
e
s


m
e
t
a
c
l
a
s
s
,
 
k
w
a
r
g
s
,
 
n
s
 
=
 
t
y
p
e
s
.
p
r
e
p
a
r
e
_
c
l
a
s
s
(
'
S
t
o
c
k
'
,
 
(
)
,
 
{
'
m
e
t
a
c
l
a
s
s
'
:
 
t
y
p
e
}
)



它会查找合适的元类并调用它的 __prepare__() 方法。
然后这个元类保存它的关键字参数，准备命名空间后被返回。

更多信息, 请参考 PEP 3115 ,
以及 Python documentation .

## 9.19 在定义的时候初始化类的成员


### 问题


你想在类被定义的时候就初始化一部分类的成员，而不是要等到实例被创建后。

### 解决方案


在类定义时就执行初始化或设置操作是元类的一个典型应用场景。本质上讲，一个元类会在定义时被触发，
这时候你可以执行一些额外的操作。

下面是一个例子，利用这个思路来创建类似于 collections 模块中的命名元组的类：

In [ ]:
i
m
p
o
r
t
 
o
p
e
r
a
t
o
r




c
l
a
s
s
 
S
t
r
u
c
t
T
u
p
l
e
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
c
l
s
,
 
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)
:


 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
i
n
i
t
_
_
(
*
a
r
g
s
,
 
*
*
k
w
a
r
g
s
)


 
 
 
 
 
 
 
 
f
o
r
 
n
,
 
n
a
m
e
 
i
n
 
e
n
u
m
e
r
a
t
e
(
c
l
s
.
_
f
i
e
l
d
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
t
a
t
t
r
(
c
l
s
,
 
n
a
m
e
,
 
p
r
o
p
e
r
t
y
(
o
p
e
r
a
t
o
r
.
i
t
e
m
g
e
t
t
e
r
(
n
)
)
)




c
l
a
s
s
 
S
t
r
u
c
t
T
u
p
l
e
(
t
u
p
l
e
,
 
m
e
t
a
c
l
a
s
s
=
S
t
r
u
c
t
T
u
p
l
e
M
e
t
a
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
]


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
i
f
 
l
e
n
(
a
r
g
s
)
 
!
=
 
l
e
n
(
c
l
s
.
_
f
i
e
l
d
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
V
a
l
u
e
E
r
r
o
r
(
'
{
}
 
a
r
g
u
m
e
n
t
s
 
r
e
q
u
i
r
e
d
'
.
f
o
r
m
a
t
(
l
e
n
(
c
l
s
.
_
f
i
e
l
d
s
)
)
)


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
u
p
e
r
(
)
.
_
_
n
e
w
_
_
(
c
l
s
,
a
r
g
s
)



这段代码可以用来定义简单的基于元组的数据结构，如下所示：

In [ ]:
c
l
a
s
s
 
S
t
o
c
k
(
S
t
r
u
c
t
T
u
p
l
e
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
'
n
a
m
e
'
,
 
'
s
h
a
r
e
s
'
,
 
'
p
r
i
c
e
'
]




c
l
a
s
s
 
P
o
i
n
t
(
S
t
r
u
c
t
T
u
p
l
e
)
:


 
 
 
 
_
f
i
e
l
d
s
 
=
 
[
'
x
'
,
 
'
y
'
]



下面演示它如何工作：

In [ ]:
s = Stock('ACME', 50, 91.1)
s

In [ ]:
s[0]

In [ ]:
s.name

In [ ]:
s.shares * s.price

In [ ]:
s.shares = 23

### 讨论


这一小节中，类 StructTupleMeta 获取到类属性 _fields 中的属性名字列表，
然后将它们转换成相应的可访问特定元组槽的方法。函数 operator.itemgetter() 创建一个访问器函数，
然后 property() 函数将其转换成一个属性。

本节最难懂的部分是知道不同的初始化步骤是什么时候发生的。
StructTupleMeta 中的 __init__() 方法只在每个类被定义时被调用一次。
cls 参数就是那个被定义的类。实际上，上述代码使用了 _fields 类变量来保存新的被定义的类，
然后给它再添加一点新的东西。

StructTuple 类作为一个普通的基类，供其他使用者来继承。
这个类中的 __new__() 方法用来构造新的实例。
这里使用 __new__() 并不是很常见，主要是因为我们要修改元组的调用签名，
使得我们可以像普通的实例调用那样创建实例。就像下面这样：

In [ ]:
s
 
=
 
S
t
o
c
k
(
'
A
C
M
E
'
,
 
5
0
,
 
9
1
.
1
)
 
#
 
O
K


s
 
=
 
S
t
o
c
k
(
(
'
A
C
M
E
'
,
 
5
0
,
 
9
1
.
1
)
)
 
#
 
E
r
r
o
r



跟 __init__() 不同的是，__new__() 方法在实例被创建之前被触发。
由于元组是不可修改的，所以一旦它们被创建了就不可能对它做任何改变。而 __init__() 会在实例创建的最后被触发，
这样的话我们就可以做我们想做的了。这也是为什么 __new__() 方法已经被定义了。

尽管本节很短，还是需要你能仔细研读，深入思考Python类是如何被定义的，实例是如何被创建的，
还有就是元类和类的各个不同的方法究竟在什么时候被调用。

PEP 422
提供了一个解决本节问题的另外一种方法。
但是，截止到我写这本书的时候，它还没被采纳和接受。
尽管如此，如果你使用的是Python 3.3或更高的版本，那么还是值得去看一下的。

## 9.20 利用函数注解实现方法重载


### 问题


你已经学过怎样使用函数参数注解，那么你可能会想利用它来实现基于类型的方法重载。
但是你不确定应该怎样去实现（或者到底行得通不）。

### 解决方案


本小节的技术是基于一个简单的技术，那就是Python允许参数注解，代码可以像下面这样写：

In [ ]:
c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
x
:
i
n
t
,
 
y
:
i
n
t
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
1
:
'
,
 
x
,
 
y
)




 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
s
:
s
t
r
,
 
n
:
i
n
t
 
=
 
0
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
2
:
'
,
 
s
,
 
n
)




s
 
=
 
S
p
a
m
(
)


s
.
b
a
r
(
2
,
 
3
)
 
#
 
P
r
i
n
t
s
 
B
a
r
 
1
:
 
2
 
3


s
.
b
a
r
(
'
h
e
l
l
o
'
)
 
#
 
P
r
i
n
t
s
 
B
a
r
 
2
:
 
h
e
l
l
o
 
0



下面是我们第一步的尝试，使用到了一个元类和描述器：

In [ ]:
#
 
m
u
l
t
i
p
l
e
.
p
y


i
m
p
o
r
t
 
i
n
s
p
e
c
t


i
m
p
o
r
t
 
t
y
p
e
s




c
l
a
s
s
 
M
u
l
t
i
M
e
t
h
o
d
:


 
 
 
 
'
'
'


 
 
 
 
R
e
p
r
e
s
e
n
t
s
 
a
 
s
i
n
g
l
e
 
m
u
l
t
i
m
e
t
h
o
d
.


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
e
t
h
o
d
s
 
=
 
{
}


 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
n
a
m
e
_
_
 
=
 
n
a
m
e




 
 
 
 
d
e
f
 
r
e
g
i
s
t
e
r
(
s
e
l
f
,
 
m
e
t
h
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
R
e
g
i
s
t
e
r
 
a
 
n
e
w
 
m
e
t
h
o
d
 
a
s
 
a
 
m
u
l
t
i
m
e
t
h
o
d


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
s
i
g
 
=
 
i
n
s
p
e
c
t
.
s
i
g
n
a
t
u
r
e
(
m
e
t
h
)




 
 
 
 
 
 
 
 
#
 
B
u
i
l
d
 
a
 
t
y
p
e
 
s
i
g
n
a
t
u
r
e
 
f
r
o
m
 
t
h
e
 
m
e
t
h
o
d
'
s
 
a
n
n
o
t
a
t
i
o
n
s


 
 
 
 
 
 
 
 
t
y
p
e
s
 
=
 
[
]


 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
,
 
p
a
r
m
 
i
n
 
s
i
g
.
p
a
r
a
m
e
t
e
r
s
.
i
t
e
m
s
(
)
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
a
m
e
 
=
=
 
'
s
e
l
f
'
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
a
r
m
.
a
n
n
o
t
a
t
i
o
n
 
i
s
 
i
n
s
p
e
c
t
.
P
a
r
a
m
e
t
e
r
.
e
m
p
t
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
A
r
g
u
m
e
n
t
 
{
}
 
m
u
s
t
 
b
e
 
a
n
n
o
t
a
t
e
d
 
w
i
t
h
 
a
 
t
y
p
e
'
.
f
o
r
m
a
t
(
n
a
m
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
p
a
r
m
.
a
n
n
o
t
a
t
i
o
n
,
 
t
y
p
e
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
'
A
r
g
u
m
e
n
t
 
{
}
 
a
n
n
o
t
a
t
i
o
n
 
m
u
s
t
 
b
e
 
a
 
t
y
p
e
'
.
f
o
r
m
a
t
(
n
a
m
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
)


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
p
a
r
m
.
d
e
f
a
u
l
t
 
i
s
 
n
o
t
 
i
n
s
p
e
c
t
.
P
a
r
a
m
e
t
e
r
.
e
m
p
t
y
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
e
t
h
o
d
s
[
t
u
p
l
e
(
t
y
p
e
s
)
]
 
=
 
m
e
t
h


 
 
 
 
 
 
 
 
 
 
 
 
t
y
p
e
s
.
a
p
p
e
n
d
(
p
a
r
m
.
a
n
n
o
t
a
t
i
o
n
)




 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
e
t
h
o
d
s
[
t
u
p
l
e
(
t
y
p
e
s
)
]
 
=
 
m
e
t
h




 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
C
a
l
l
 
a
 
m
e
t
h
o
d
 
b
a
s
e
d
 
o
n
 
t
y
p
e
 
s
i
g
n
a
t
u
r
e
 
o
f
 
t
h
e
 
a
r
g
u
m
e
n
t
s


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
t
y
p
e
s
 
=
 
t
u
p
l
e
(
t
y
p
e
(
a
r
g
)
 
f
o
r
 
a
r
g
 
i
n
 
a
r
g
s
[
1
:
]
)


 
 
 
 
 
 
 
 
m
e
t
h
 
=
 
s
e
l
f
.
_
m
e
t
h
o
d
s
.
g
e
t
(
t
y
p
e
s
,
 
N
o
n
e
)


 
 
 
 
 
 
 
 
i
f
 
m
e
t
h
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
m
e
t
h
(
*
a
r
g
s
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
N
o
 
m
a
t
c
h
i
n
g
 
m
e
t
h
o
d
 
f
o
r
 
t
y
p
e
s
 
{
}
'
.
f
o
r
m
a
t
(
t
y
p
e
s
)
)




 
 
 
 
d
e
f
 
_
_
g
e
t
_
_
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
,
 
c
l
s
)
:


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
D
e
s
c
r
i
p
t
o
r
 
m
e
t
h
o
d
 
n
e
e
d
e
d
 
t
o
 
m
a
k
e
 
c
a
l
l
s
 
w
o
r
k
 
i
n
 
a
 
c
l
a
s
s


 
 
 
 
 
 
 
 
'
'
'


 
 
 
 
 
 
 
 
i
f
 
i
n
s
t
a
n
c
e
 
i
s
 
n
o
t
 
N
o
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
s
.
M
e
t
h
o
d
T
y
p
e
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f




c
l
a
s
s
 
M
u
l
t
i
D
i
c
t
(
d
i
c
t
)
:


 
 
 
 
'
'
'


 
 
 
 
S
p
e
c
i
a
l
 
d
i
c
t
i
o
n
a
r
y
 
t
o
 
b
u
i
l
d
 
m
u
l
t
i
m
e
t
h
o
d
s
 
i
n
 
a
 
m
e
t
a
c
l
a
s
s


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
_
_
s
e
t
i
t
e
m
_
_
(
s
e
l
f
,
 
k
e
y
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
k
e
y
 
i
n
 
s
e
l
f
:


 
 
 
 
 
 
 
 
 
 
 
 
#
 
I
f
 
k
e
y
 
a
l
r
e
a
d
y
 
e
x
i
s
t
s
,
 
i
t
 
m
u
s
t
 
b
e
 
a
 
m
u
l
t
i
m
e
t
h
o
d
 
o
r
 
c
a
l
l
a
b
l
e


 
 
 
 
 
 
 
 
 
 
 
 
c
u
r
r
e
n
t
_
v
a
l
u
e
 
=
 
s
e
l
f
[
k
e
y
]


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
c
u
r
r
e
n
t
_
v
a
l
u
e
,
 
M
u
l
t
i
M
e
t
h
o
d
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
u
r
r
e
n
t
_
v
a
l
u
e
.
r
e
g
i
s
t
e
r
(
v
a
l
u
e
)


 
 
 
 
 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
v
a
l
u
e
 
=
 
M
u
l
t
i
M
e
t
h
o
d
(
k
e
y
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
v
a
l
u
e
.
r
e
g
i
s
t
e
r
(
c
u
r
r
e
n
t
_
v
a
l
u
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
m
v
a
l
u
e
.
r
e
g
i
s
t
e
r
(
v
a
l
u
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
s
e
t
i
t
e
m
_
_
(
k
e
y
,
 
m
v
a
l
u
e
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
s
u
p
e
r
(
)
.
_
_
s
e
t
i
t
e
m
_
_
(
k
e
y
,
 
v
a
l
u
e
)




c
l
a
s
s
 
M
u
l
t
i
p
l
e
M
e
t
a
(
t
y
p
e
)
:


 
 
 
 
'
'
'


 
 
 
 
M
e
t
a
c
l
a
s
s
 
t
h
a
t
 
a
l
l
o
w
s
 
m
u
l
t
i
p
l
e
 
d
i
s
p
a
t
c
h
 
o
f
 
m
e
t
h
o
d
s


 
 
 
 
'
'
'


 
 
 
 
d
e
f
 
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
c
l
s
d
i
c
t
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
.
_
_
n
e
w
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
,
 
d
i
c
t
(
c
l
s
d
i
c
t
)
)




 
 
 
 
@
c
l
a
s
s
m
e
t
h
o
d


 
 
 
 
d
e
f
 
_
_
p
r
e
p
a
r
e
_
_
(
c
l
s
,
 
c
l
s
n
a
m
e
,
 
b
a
s
e
s
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
M
u
l
t
i
D
i
c
t
(
)



为了使用这个类，你可以像下面这样写：

In [ ]:
c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
M
u
l
t
i
p
l
e
M
e
t
a
)
:


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
x
:
i
n
t
,
 
y
:
i
n
t
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
1
:
'
,
 
x
,
 
y
)




 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
s
:
s
t
r
,
 
n
:
i
n
t
 
=
 
0
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
2
:
'
,
 
s
,
 
n
)




#
 
E
x
a
m
p
l
e
:
 
o
v
e
r
l
o
a
d
e
d
 
_
_
i
n
i
t
_
_


i
m
p
o
r
t
 
t
i
m
e




c
l
a
s
s
 
D
a
t
e
(
m
e
t
a
c
l
a
s
s
=
M
u
l
t
i
p
l
e
M
e
t
a
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
y
e
a
r
:
 
i
n
t
,
 
m
o
n
t
h
:
i
n
t
,
 
d
a
y
:
i
n
t
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
y
e
a
r
 
=
 
y
e
a
r


 
 
 
 
 
 
 
 
s
e
l
f
.
m
o
n
t
h
 
=
 
m
o
n
t
h


 
 
 
 
 
 
 
 
s
e
l
f
.
d
a
y
 
=
 
d
a
y




 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
t
 
=
 
t
i
m
e
.
l
o
c
a
l
t
i
m
e
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
i
n
i
t
_
_
(
t
.
t
m
_
y
e
a
r
,
 
t
.
t
m
_
m
o
n
,
 
t
.
t
m
_
m
d
a
y
)



下面是一个交互示例来验证它能正确的工作：

In [ ]:
s = Spam()
s.bar(2, 3)

In [ ]:
s.bar('hello')

In [ ]:
s.bar('hello', 5)

In [ ]:
s.bar(2, 'hello')

In [ ]:
# Overloaded __init__
d = Date(2012, 12, 21)
# Get today's date
e = Date()
e.year

In [ ]:
e.month

In [ ]:
e.day

### 讨论


坦白来讲，相对于通常的代码而已本节使用到了很多的魔法代码。
但是，它却能让我们深入理解元类和描述器的底层工作原理，
并能加深对这些概念的印象。因此，就算你并不会立即去应用本节的技术，
它的一些底层思想却会影响到其它涉及到元类、描述器和函数注解的编程技术。

本节的实现中的主要思路其实是很简单的。MutipleMeta 元类使用它的 __prepare__() 方法
来提供一个作为 MultiDict 实例的自定义字典。这个跟普通字典不一样的是，
MultiDict 会在元素被设置的时候检查是否已经存在，如果存在的话，重复的元素会在 MultiMethod
实例中合并。

MultiMethod 实例通过构建从类型签名到函数的映射来收集方法。
在这个构建过程中，函数注解被用来收集这些签名然后构建这个映射。
这个过程在 MultiMethod.register() 方法中实现。
这种映射的一个关键特点是对于多个方法，所有参数类型都必须要指定，否则就会报错。

为了让 MultiMethod 实例模拟一个调用，它的 __call__() 方法被实现了。
这个方法从所有排除 slef 的参数中构建一个类型元组，在内部map中查找这个方法，
然后调用相应的方法。为了能让 MultiMethod 实例在类定义时正确操作，__get__() 是必须得实现的。
它被用来构建正确的绑定方法。比如：

In [ ]:
b = s.bar
b

In [ ]:
b.__self__

In [ ]:
b.__func__

In [ ]:
b(2, 3)

In [ ]:
b('hello')

不过本节的实现还有一些限制，其中一个是它不能使用关键字参数。例如：

In [ ]:
s.bar(x=2, y=3)

In [ ]:
s.bar(s='hello')

也许有其他的方法能添加这种支持，但是它需要一个完全不同的方法映射方式。
问题在于关键字参数的出现是没有顺序的。当它跟位置参数混合使用时，
那你的参数就会变得比较混乱了，这时候你不得不在 __call__() 方法中先去做个排序。

同样对于继承也是有限制的，例如，类似下面这种代码就不能正常工作：

In [ ]:
c
l
a
s
s
 
A
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
B
(
A
)
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
C
:


 
 
 
 
p
a
s
s




c
l
a
s
s
 
S
p
a
m
(
m
e
t
a
c
l
a
s
s
=
M
u
l
t
i
p
l
e
M
e
t
a
)
:


 
 
 
 
d
e
f
 
f
o
o
(
s
e
l
f
,
 
x
:
A
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
F
o
o
 
1
:
'
,
 
x
)




 
 
 
 
d
e
f
 
f
o
o
(
s
e
l
f
,
 
x
:
C
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
F
o
o
 
2
:
'
,
 
x
)



原因是因为 x:A 注解不能成功匹配子类实例（比如B的实例），如下：

In [ ]:
s = Spam()
a = A()
s.foo(a)

In [ ]:
c = C()
s.foo(c)

In [ ]:
b = B()
s.foo(b)

作为使用元类和注解的一种替代方案，可以通过描述器来实现类似的效果。例如：

In [ ]:
i
m
p
o
r
t
 
t
y
p
e
s




c
l
a
s
s
 
m
u
l
t
i
m
e
t
h
o
d
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
f
u
n
c
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
e
t
h
o
d
s
 
=
 
{
}


 
 
 
 
 
 
 
 
s
e
l
f
.
_
_
n
a
m
e
_
_
 
=
 
f
u
n
c
.
_
_
n
a
m
e
_
_


 
 
 
 
 
 
 
 
s
e
l
f
.
_
d
e
f
a
u
l
t
 
=
 
f
u
n
c




 
 
 
 
d
e
f
 
m
a
t
c
h
(
s
e
l
f
,
 
*
t
y
p
e
s
)
:


 
 
 
 
 
 
 
 
d
e
f
 
r
e
g
i
s
t
e
r
(
f
u
n
c
)
:


 
 
 
 
 
 
 
 
 
 
 
 
n
d
e
f
a
u
l
t
s
 
=
 
l
e
n
(
f
u
n
c
.
_
_
d
e
f
a
u
l
t
s
_
_
)
 
i
f
 
f
u
n
c
.
_
_
d
e
f
a
u
l
t
s
_
_
 
e
l
s
e
 
0


 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
 
i
n
 
r
a
n
g
e
(
n
d
e
f
a
u
l
t
s
+
1
)
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
_
m
e
t
h
o
d
s
[
t
y
p
e
s
[
:
l
e
n
(
t
y
p
e
s
)
 
-
 
n
]
]
 
=
 
f
u
n
c


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
r
e
g
i
s
t
e
r




 
 
 
 
d
e
f
 
_
_
c
a
l
l
_
_
(
s
e
l
f
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
t
y
p
e
s
 
=
 
t
u
p
l
e
(
t
y
p
e
(
a
r
g
)
 
f
o
r
 
a
r
g
 
i
n
 
a
r
g
s
[
1
:
]
)


 
 
 
 
 
 
 
 
m
e
t
h
 
=
 
s
e
l
f
.
_
m
e
t
h
o
d
s
.
g
e
t
(
t
y
p
e
s
,
 
N
o
n
e
)


 
 
 
 
 
 
 
 
i
f
 
m
e
t
h
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
m
e
t
h
(
*
a
r
g
s
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
d
e
f
a
u
l
t
(
*
a
r
g
s
)




 
 
 
 
d
e
f
 
_
_
g
e
t
_
_
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
,
 
c
l
s
)
:


 
 
 
 
 
 
 
 
i
f
 
i
n
s
t
a
n
c
e
 
i
s
 
n
o
t
 
N
o
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
t
y
p
e
s
.
M
e
t
h
o
d
T
y
p
e
(
s
e
l
f
,
 
i
n
s
t
a
n
c
e
)


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f



为了使用描述器版本，你需要像下面这样写：

In [ ]:
c
l
a
s
s
 
S
p
a
m
:


 
 
 
 
@
m
u
l
t
i
m
e
t
h
o
d


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
*
a
r
g
s
)
:


 
 
 
 
 
 
 
 
#
 
D
e
f
a
u
l
t
 
m
e
t
h
o
d
 
c
a
l
l
e
d
 
i
f
 
n
o
 
m
a
t
c
h


 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
N
o
 
m
a
t
c
h
i
n
g
 
m
e
t
h
o
d
 
f
o
r
 
b
a
r
'
)




 
 
 
 
@
b
a
r
.
m
a
t
c
h
(
i
n
t
,
 
i
n
t
)


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
x
,
 
y
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
1
:
'
,
 
x
,
 
y
)




 
 
 
 
@
b
a
r
.
m
a
t
c
h
(
s
t
r
,
 
i
n
t
)


 
 
 
 
d
e
f
 
b
a
r
(
s
e
l
f
,
 
s
,
 
n
 
=
 
0
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
B
a
r
 
2
:
'
,
 
s
,
 
n
)



描述器方案同样也有前面提到的限制（不支持关键字参数和继承）。

所有事物都是平等的，有好有坏，也许最好的办法就是在普通代码中避免使用方法重载。
不过有些特殊情况下还是有意义的，比如基于模式匹配的方法重载程序中。
举个例子，8.21小节中的访问者模式可以修改为一个使用方法重载的类。
但是，除了这个以外，通常不应该使用方法重载（就简单的使用不同名称的方法就行了）。

在Python社区对于实现方法重载的讨论已经由来已久。
对于引发这个争论的原因，可以参考下Guido van Rossum的这篇博客：
Five-Minute Multimethods in Python

## 9.21 避免重复的属性方法


### 问题


你在类中需要重复的定义一些执行相同逻辑的属性方法，比如进行类型检查，怎样去简化这些重复代码呢？

### 解决方案


考虑下一个简单的类，它的属性由属性方法包装：

In [ ]:
c
l
a
s
s
 
P
e
r
s
o
n
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
 
,
a
g
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
 
 
 
 
s
e
l
f
.
a
g
e
 
=
 
a
g
e




 
 
 
 
@
p
r
o
p
e
r
t
y


 
 
 
 
d
e
f
 
n
a
m
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
n
a
m
e




 
 
 
 
@
n
a
m
e
.
s
e
t
t
e
r


 
 
 
 
d
e
f
 
n
a
m
e
(
s
e
l
f
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
s
t
r
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
n
a
m
e
 
m
u
s
t
 
b
e
 
a
 
s
t
r
i
n
g
'
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
n
a
m
e
 
=
 
v
a
l
u
e




 
 
 
 
@
p
r
o
p
e
r
t
y


 
 
 
 
d
e
f
 
a
g
e
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
s
e
l
f
.
_
a
g
e




 
 
 
 
@
a
g
e
.
s
e
t
t
e
r


 
 
 
 
d
e
f
 
a
g
e
(
s
e
l
f
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
i
n
t
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
a
g
e
 
m
u
s
t
 
b
e
 
a
n
 
i
n
t
'
)


 
 
 
 
 
 
 
 
s
e
l
f
.
_
a
g
e
 
=
 
v
a
l
u
e



可以看到，为了实现属性值的类型检查我们写了很多的重复代码。
只要你以后看到类似这样的代码，你都应该想办法去简化它。
一个可行的方法是创建一个函数用来定义属性并返回它。例如：

In [ ]:
d
e
f
 
t
y
p
e
d
_
p
r
o
p
e
r
t
y
(
n
a
m
e
,
 
e
x
p
e
c
t
e
d
_
t
y
p
e
)
:


 
 
 
 
s
t
o
r
a
g
e
_
n
a
m
e
 
=
 
'
_
'
 
+
 
n
a
m
e




 
 
 
 
@
p
r
o
p
e
r
t
y


 
 
 
 
d
e
f
 
p
r
o
p
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
g
e
t
a
t
t
r
(
s
e
l
f
,
 
s
t
o
r
a
g
e
_
n
a
m
e
)




 
 
 
 
@
p
r
o
p
.
s
e
t
t
e
r


 
 
 
 
d
e
f
 
p
r
o
p
(
s
e
l
f
,
 
v
a
l
u
e
)
:


 
 
 
 
 
 
 
 
i
f
 
n
o
t
 
i
s
i
n
s
t
a
n
c
e
(
v
a
l
u
e
,
 
e
x
p
e
c
t
e
d
_
t
y
p
e
)
:


 
 
 
 
 
 
 
 
 
 
 
 
r
a
i
s
e
 
T
y
p
e
E
r
r
o
r
(
'
{
}
 
m
u
s
t
 
b
e
 
a
 
{
}
'
.
f
o
r
m
a
t
(
n
a
m
e
,
 
e
x
p
e
c
t
e
d
_
t
y
p
e
)
)


 
 
 
 
 
 
 
 
s
e
t
a
t
t
r
(
s
e
l
f
,
 
s
t
o
r
a
g
e
_
n
a
m
e
,
 
v
a
l
u
e
)




 
 
 
 
r
e
t
u
r
n
 
p
r
o
p




#
 
E
x
a
m
p
l
e
 
u
s
e


c
l
a
s
s
 
P
e
r
s
o
n
:


 
 
 
 
n
a
m
e
 
=
 
t
y
p
e
d
_
p
r
o
p
e
r
t
y
(
'
n
a
m
e
'
,
 
s
t
r
)


 
 
 
 
a
g
e
 
=
 
t
y
p
e
d
_
p
r
o
p
e
r
t
y
(
'
a
g
e
'
,
 
i
n
t
)




 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
a
g
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
 
 
 
 
s
e
l
f
.
a
g
e
 
=
 
a
g
e



### 讨论


本节我们演示内部函数或者闭包的一个重要特性，它们很像一个宏。例子中的函数 typed_property()
看上去有点难理解，其实它所做的仅仅就是为你生成属性并返回这个属性对象。
因此，当在一个类中使用它的时候，效果跟将它里面的代码放到类定义中去是一样的。
尽管属性的 getter 和 setter 方法访问了本地变量如 name , expected_type
以及 storate_name ，这个很正常，这些变量的值会保存在闭包当中。

我们还可以使用 functools.partial() 来稍稍改变下这个例子，很有趣。例如，你可以像下面这样：

In [ ]:
f
r
o
m
 
f
u
n
c
t
o
o
l
s
 
i
m
p
o
r
t
 
p
a
r
t
i
a
l




S
t
r
i
n
g
 
=
 
p
a
r
t
i
a
l
(
t
y
p
e
d
_
p
r
o
p
e
r
t
y
,
 
e
x
p
e
c
t
e
d
_
t
y
p
e
=
s
t
r
)


I
n
t
e
g
e
r
 
=
 
p
a
r
t
i
a
l
(
t
y
p
e
d
_
p
r
o
p
e
r
t
y
,
 
e
x
p
e
c
t
e
d
_
t
y
p
e
=
i
n
t
)




#
 
E
x
a
m
p
l
e
:


c
l
a
s
s
 
P
e
r
s
o
n
:


 
 
 
 
n
a
m
e
 
=
 
S
t
r
i
n
g
(
'
n
a
m
e
'
)


 
 
 
 
a
g
e
 
=
 
I
n
t
e
g
e
r
(
'
a
g
e
'
)




 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
n
a
m
e
,
 
a
g
e
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
n
a
m
e
 
=
 
n
a
m
e


 
 
 
 
 
 
 
 
s
e
l
f
.
a
g
e
 
=
 
a
g
e



其实你可以发现，这里的代码跟8.13小节中的类型系统描述器代码有些相似。

## 9.22 定义上下文管理器的简单方法


### 问题


你想自己去实现一个新的上下文管理器，以便使用with语句。

### 解决方案


实现一个新的上下文管理器的最简单的方法就是使用 contexlib 模块中的 @contextmanager 装饰器。
下面是一个实现了代码块计时功能的上下文管理器例子：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e


f
r
o
m
 
c
o
n
t
e
x
t
l
i
b
 
i
m
p
o
r
t
 
c
o
n
t
e
x
t
m
a
n
a
g
e
r




@
c
o
n
t
e
x
t
m
a
n
a
g
e
r


d
e
f
 
t
i
m
e
t
h
i
s
(
l
a
b
e
l
)
:


 
 
 
 
s
t
a
r
t
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
t
r
y
:


 
 
 
 
 
 
 
 
y
i
e
l
d


 
 
 
 
f
i
n
a
l
l
y
:


 
 
 
 
 
 
 
 
e
n
d
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
{
}
:
 
{
}
'
.
f
o
r
m
a
t
(
l
a
b
e
l
,
 
e
n
d
 
-
 
s
t
a
r
t
)
)




#
 
E
x
a
m
p
l
e
 
u
s
e


w
i
t
h
 
t
i
m
e
t
h
i
s
(
'
c
o
u
n
t
i
n
g
'
)
:


 
 
 
 
n
 
=
 
1
0
0
0
0
0
0
0


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
n
 
-
=
 
1



在函数 timethis() 中，yield 之前的代码会在上下文管理器中作为 __enter__() 方法执行，
所有在 yield 之后的代码会作为 __exit__() 方法执行。
如果出现了异常，异常会在yield语句那里抛出。

下面是一个更加高级一点的上下文管理器，实现了列表对象上的某种事务：

In [ ]:
@
c
o
n
t
e
x
t
m
a
n
a
g
e
r


d
e
f
 
l
i
s
t
_
t
r
a
n
s
a
c
t
i
o
n
(
o
r
i
g
_
l
i
s
t
)
:


 
 
 
 
w
o
r
k
i
n
g
 
=
 
l
i
s
t
(
o
r
i
g
_
l
i
s
t
)


 
 
 
 
y
i
e
l
d
 
w
o
r
k
i
n
g


 
 
 
 
o
r
i
g
_
l
i
s
t
[
:
]
 
=
 
w
o
r
k
i
n
g



这段代码的作用是任何对列表的修改只有当所有代码运行完成并且不出现异常的情况下才会生效。
下面我们来演示一下：

In [ ]:
items = [1, 2, 3]
with list_transaction(items) as working:
    working.append(4)
    working.append(5)
items

In [ ]:
with list_transaction(items) as working:
    working.append(6)
    working.append(7)
    raise RuntimeError('oops')

In [ ]:
items

### 讨论


通常情况下，如果要写一个上下文管理器，你需要定义一个类，里面包含一个 __enter__() 和一个
__exit__() 方法，如下所示：

In [ ]:
i
m
p
o
r
t
 
t
i
m
e




c
l
a
s
s
 
t
i
m
e
t
h
i
s
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
l
a
b
e
l
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
l
a
b
e
l
 
=
 
l
a
b
e
l




 
 
 
 
d
e
f
 
_
_
e
n
t
e
r
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
s
t
a
r
t
 
=
 
t
i
m
e
.
t
i
m
e
(
)




 
 
 
 
d
e
f
 
_
_
e
x
i
t
_
_
(
s
e
l
f
,
 
e
x
c
_
t
y
,
 
e
x
c
_
v
a
l
,
 
e
x
c
_
t
b
)
:


 
 
 
 
 
 
 
 
e
n
d
 
=
 
t
i
m
e
.
t
i
m
e
(
)


 
 
 
 
 
 
 
 
p
r
i
n
t
(
'
{
}
:
 
{
}
'
.
f
o
r
m
a
t
(
s
e
l
f
.
l
a
b
e
l
,
 
e
n
d
 
-
 
s
e
l
f
.
s
t
a
r
t
)
)



尽管这个也不难写，但是相比较写一个简单的使用 @contextmanager 注解的函数而言还是稍显乏味。

@contextmanager 应该仅仅用来写自包含的上下文管理函数。
如果你有一些对象(比如一个文件、网络连接或锁)，需要支持 with 语句，那么你就需要单独实现
__enter__() 方法和 __exit__() 方法。

## 9.23 在局部变量域中执行代码


### 问题


你想在使用范围内执行某个代码片段，并且希望在执行后所有的结果都不可见。

### 解决方案


为了理解这个问题，先试试一个简单场景。首先，在全局命名空间内执行一个代码片段：

In [ ]:
a = 13
exec('b = a + 1')
print(b)

然后，再在一个函数中执行同样的代码：

In [ ]:
def test():
    a = 13
    exec('b = a + 1')
    print(b)
test()

可以看出，最后抛出了一个NameError异常，就跟在 exec() 语句从没执行过一样。
要是你想在后面的计算中使用到 exec() 执行结果的话就会有问题了。

为了修正这样的错误，你需要在调用 exec() 之前使用 locals() 函数来得到一个局部变量字典。
之后你就能从局部字典中获取修改过后的变量值了。例如：

In [ ]:
def test():
    a = 13
    loc = locals()
    exec('b = a + 1')
    b = loc['b']
    print(b)
test()

### 讨论


实际上对于 exec() 的正确使用是比较难的。大多数情况下当你要考虑使用 exec() 的时候，
还有另外更好的解决方案（比如装饰器、闭包、元类等等）。

然而，如果你仍然要使用 exec() ，本节列出了一些如何正确使用它的方法。
默认情况下，exec() 会在调用者局部和全局范围内执行代码。然而，在函数里面，
传递给 exec() 的局部范围是拷贝实际局部变量组成的一个字典。
因此，如果 exec() 如果执行了修改操作，这种修改后的结果对实际局部变量值是没有影响的。
下面是另外一个演示它的例子：

In [ ]:
def test1():
    x = 0
    exec('x += 1')
    print(x)
test1()

上面代码里，当你调用 locals() 获取局部变量时，你获得的是传递给 exec() 的局部变量的一个拷贝。
通过在代码执行后审查这个字典的值，那就能获取修改后的值了。下面是一个演示例子：

In [ ]:
def test2():
    x = 0
    loc = locals()
    print('before:', loc)
    exec('x += 1')
    print('after:', loc)
    print('x =', x)
test2()

仔细观察最后一步的输出，除非你将 loc 中被修改后的值手动赋值给x，否则x变量值是不会变的。

在使用 locals() 的时候，你需要注意操作顺序。每次它被调用的时候，
locals() 会获取局部变量值中的值并覆盖字典中相应的变量。
请注意观察下下面这个试验的输出结果：

In [ ]:
def test3():
    x = 0
    loc = locals()
    print(loc)
    exec('x += 1')
    print(loc)
    locals()
    print(loc)
test3()

注意最后一次调用 locals() 的时候x的值是如何被覆盖掉的。

作为 locals() 的一个替代方案，你可以使用你自己的字典，并将它传递给 exec() 。例如：

In [ ]:
def test4():
    a = 13
    loc = { 'a' : a }
    glb = { }
    exec('b = a + 1', glb, loc)
    b = loc['b']
    print(b)
test4()

大部分情况下，这种方式是使用 exec() 的最佳实践。
你只需要保证全局和局部字典在后面代码访问时已经被初始化。

还有一点，在使用 exec() 之前，你可能需要问下自己是否有其他更好的替代方案。
大多数情况下当你要考虑使用 exec() 的时候，
还有另外更好的解决方案，比如装饰器、闭包、元类，或其他一些元编程特性。

## 9.24 解析与分析Python源码


### 问题


你想写解析并分析Python源代码的程序。

### 解决方案


大部分程序员知道Python能够计算或执行字符串形式的源代码。例如：

In [ ]:
x = 42
eval('2 + 3*4 + x')

In [ ]:
exec('for i in range(10): print(i)')

尽管如此，ast 模块能被用来将Python源码编译成一个可被分析的抽象语法树（AST）。例如：

In [ ]:
import ast
ex = ast.parse('2 + 3*4 + x', mode='eval')
ex

In [ ]:
ast.dump(ex)

In [ ]:
top = ast.parse('for i in range(10): print(i)', mode='exec')
top

In [ ]:
ast.dump(top)

分析源码树需要你自己更多的学习，它是由一系列AST节点组成的。
分析这些节点最简单的方法就是定义一个访问者类，实现很多 visit_NodeName() 方法，
NodeName() 匹配那些你感兴趣的节点。下面是这样一个类，记录了哪些名字被加载、存储和删除的信息。

In [ ]:
i
m
p
o
r
t
 
a
s
t




c
l
a
s
s
 
C
o
d
e
A
n
a
l
y
z
e
r
(
a
s
t
.
N
o
d
e
V
i
s
i
t
o
r
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
a
d
e
d
 
=
 
s
e
t
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
s
t
o
r
e
d
 
=
 
s
e
t
(
)


 
 
 
 
 
 
 
 
s
e
l
f
.
d
e
l
e
t
e
d
 
=
 
s
e
t
(
)




 
 
 
 
d
e
f
 
v
i
s
i
t
_
N
a
m
e
(
s
e
l
f
,
 
n
o
d
e
)
:


 
 
 
 
 
 
 
 
i
f
 
i
s
i
n
s
t
a
n
c
e
(
n
o
d
e
.
c
t
x
,
 
a
s
t
.
L
o
a
d
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
a
d
e
d
.
a
d
d
(
n
o
d
e
.
i
d
)


 
 
 
 
 
 
 
 
e
l
i
f
 
i
s
i
n
s
t
a
n
c
e
(
n
o
d
e
.
c
t
x
,
 
a
s
t
.
S
t
o
r
e
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
s
t
o
r
e
d
.
a
d
d
(
n
o
d
e
.
i
d
)


 
 
 
 
 
 
 
 
e
l
i
f
 
i
s
i
n
s
t
a
n
c
e
(
n
o
d
e
.
c
t
x
,
 
a
s
t
.
D
e
l
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
e
l
f
.
d
e
l
e
t
e
d
.
a
d
d
(
n
o
d
e
.
i
d
)




#
 
S
a
m
p
l
e
 
u
s
a
g
e


i
f
 
_
_
n
a
m
e
_
_
 
=
=
 
'
_
_
m
a
i
n
_
_
'
:


 
 
 
 
#
 
S
o
m
e
 
P
y
t
h
o
n
 
c
o
d
e


 
 
 
 
c
o
d
e
 
=
 
'
'
'


 
 
 
 
f
o
r
 
i
 
i
n
 
r
a
n
g
e
(
1
0
)
:


 
 
 
 
 
 
 
 
p
r
i
n
t
(
i
)


 
 
 
 
d
e
l
 
i


 
 
 
 
'
'
'




 
 
 
 
#
 
P
a
r
s
e
 
i
n
t
o
 
a
n
 
A
S
T


 
 
 
 
t
o
p
 
=
 
a
s
t
.
p
a
r
s
e
(
c
o
d
e
,
 
m
o
d
e
=
'
e
x
e
c
'
)




 
 
 
 
#
 
F
e
e
d
 
t
h
e
 
A
S
T
 
t
o
 
a
n
a
l
y
z
e
 
n
a
m
e
 
u
s
a
g
e


 
 
 
 
c
 
=
 
C
o
d
e
A
n
a
l
y
z
e
r
(
)


 
 
 
 
c
.
v
i
s
i
t
(
t
o
p
)


 
 
 
 
p
r
i
n
t
(
'
L
o
a
d
e
d
:
'
,
 
c
.
l
o
a
d
e
d
)


 
 
 
 
p
r
i
n
t
(
'
S
t
o
r
e
d
:
'
,
 
c
.
s
t
o
r
e
d
)


 
 
 
 
p
r
i
n
t
(
'
D
e
l
e
t
e
d
:
'
,
 
c
.
d
e
l
e
t
e
d
)



如果你运行这个程序，你会得到下面这样的输出：

In [ ]:
L
o
a
d
e
d
:
 
{
'
i
'
,
 
'
r
a
n
g
e
'
,
 
'
p
r
i
n
t
'
}


S
t
o
r
e
d
:
 
{
'
i
'
}


D
e
l
e
t
e
d
:
 
{
'
i
'
}



最后，AST可以通过 compile() 函数来编译并执行。例如：

In [ ]:
exec(compile(top,'<stdin>', 'exec'))

### 讨论


当你能够分析源代码并从中获取信息的时候，你就能写很多代码分析、优化或验证工具了。
例如，相比盲目的传递一些代码片段到类似 exec() 函数中，你可以先将它转换成一个AST，
然后观察它的细节看它到底是怎样做的。
你还可以写一些工具来查看某个模块的全部源码，并且在此基础上执行某些静态分析。

需要注意的是，如果你知道自己在干啥，你还能够重写AST来表示新的代码。
下面是一个装饰器例子，可以通过重新解析函数体源码、
重写AST并重新创建函数代码对象来将全局访问变量降为函数体作用范围，

In [ ]:
#
 
n
a
m
e
l
o
w
e
r
.
p
y


i
m
p
o
r
t
 
a
s
t


i
m
p
o
r
t
 
i
n
s
p
e
c
t




#
 
N
o
d
e
 
v
i
s
i
t
o
r
 
t
h
a
t
 
l
o
w
e
r
s
 
g
l
o
b
a
l
l
y
 
a
c
c
e
s
s
e
d
 
n
a
m
e
s
 
i
n
t
o


#
 
t
h
e
 
f
u
n
c
t
i
o
n
 
b
o
d
y
 
a
s
 
l
o
c
a
l
 
v
a
r
i
a
b
l
e
s
.


c
l
a
s
s
 
N
a
m
e
L
o
w
e
r
(
a
s
t
.
N
o
d
e
V
i
s
i
t
o
r
)
:


 
 
 
 
d
e
f
 
_
_
i
n
i
t
_
_
(
s
e
l
f
,
 
l
o
w
e
r
e
d
_
n
a
m
e
s
)
:


 
 
 
 
 
 
 
 
s
e
l
f
.
l
o
w
e
r
e
d
_
n
a
m
e
s
 
=
 
l
o
w
e
r
e
d
_
n
a
m
e
s




 
 
 
 
d
e
f
 
v
i
s
i
t
_
F
u
n
c
t
i
o
n
D
e
f
(
s
e
l
f
,
 
n
o
d
e
)
:


 
 
 
 
 
 
 
 
#
 
C
o
m
p
i
l
e
 
s
o
m
e
 
a
s
s
i
g
n
m
e
n
t
s
 
t
o
 
l
o
w
e
r
 
t
h
e
 
c
o
n
s
t
a
n
t
s


 
 
 
 
 
 
 
 
c
o
d
e
 
=
 
'
_
_
g
l
o
b
a
l
s
 
=
 
g
l
o
b
a
l
s
(
)
\
n
'


 
 
 
 
 
 
 
 
c
o
d
e
 
+
=
 
'
\
n
'
.
j
o
i
n
(
"
{
0
}
 
=
 
_
_
g
l
o
b
a
l
s
[
'
{
0
}
'
]
"
.
f
o
r
m
a
t
(
n
a
m
e
)


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
f
o
r
 
n
a
m
e
 
i
n
 
s
e
l
f
.
l
o
w
e
r
e
d
_
n
a
m
e
s
)


 
 
 
 
 
 
 
 
c
o
d
e
_
a
s
t
 
=
 
a
s
t
.
p
a
r
s
e
(
c
o
d
e
,
 
m
o
d
e
=
'
e
x
e
c
'
)




 
 
 
 
 
 
 
 
#
 
I
n
j
e
c
t
 
n
e
w
 
s
t
a
t
e
m
e
n
t
s
 
i
n
t
o
 
t
h
e
 
f
u
n
c
t
i
o
n
 
b
o
d
y


 
 
 
 
 
 
 
 
n
o
d
e
.
b
o
d
y
[
:
0
]
 
=
 
c
o
d
e
_
a
s
t
.
b
o
d
y




 
 
 
 
 
 
 
 
#
 
S
a
v
e
 
t
h
e
 
f
u
n
c
t
i
o
n
 
o
b
j
e
c
t


 
 
 
 
 
 
 
 
s
e
l
f
.
f
u
n
c
 
=
 
n
o
d
e




#
 
D
e
c
o
r
a
t
o
r
 
t
h
a
t
 
t
u
r
n
s
 
g
l
o
b
a
l
 
n
a
m
e
s
 
i
n
t
o
 
l
o
c
a
l
s


d
e
f
 
l
o
w
e
r
_
n
a
m
e
s
(
*
n
a
m
e
l
i
s
t
)
:


 
 
 
 
d
e
f
 
l
o
w
e
r
(
f
u
n
c
)
:


 
 
 
 
 
 
 
 
s
r
c
l
i
n
e
s
 
=
 
i
n
s
p
e
c
t
.
g
e
t
s
o
u
r
c
e
(
f
u
n
c
)
.
s
p
l
i
t
l
i
n
e
s
(
)


 
 
 
 
 
 
 
 
#
 
S
k
i
p
 
s
o
u
r
c
e
 
l
i
n
e
s
 
p
r
i
o
r
 
t
o
 
t
h
e
 
@
l
o
w
e
r
_
n
a
m
e
s
 
d
e
c
o
r
a
t
o
r


 
 
 
 
 
 
 
 
f
o
r
 
n
,
 
l
i
n
e
 
i
n
 
e
n
u
m
e
r
a
t
e
(
s
r
c
l
i
n
e
s
)
:


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
'
@
l
o
w
e
r
_
n
a
m
e
s
'
 
i
n
 
l
i
n
e
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
b
r
e
a
k




 
 
 
 
 
 
 
 
s
r
c
 
=
 
'
\
n
'
.
j
o
i
n
(
s
r
c
l
i
n
e
s
[
n
+
1
:
]
)


 
 
 
 
 
 
 
 
#
 
H
a
c
k
 
t
o
 
d
e
a
l
 
w
i
t
h
 
i
n
d
e
n
t
e
d
 
c
o
d
e


 
 
 
 
 
 
 
 
i
f
 
s
r
c
.
s
t
a
r
t
s
w
i
t
h
(
(
'
 
'
,
'
\
t
'
)
)
:


 
 
 
 
 
 
 
 
 
 
 
 
s
r
c
 
=
 
'
i
f
 
1
:
\
n
'
 
+
 
s
r
c


 
 
 
 
 
 
 
 
t
o
p
 
=
 
a
s
t
.
p
a
r
s
e
(
s
r
c
,
 
m
o
d
e
=
'
e
x
e
c
'
)




 
 
 
 
 
 
 
 
#
 
T
r
a
n
s
f
o
r
m
 
t
h
e
 
A
S
T


 
 
 
 
 
 
 
 
c
l
 
=
 
N
a
m
e
L
o
w
e
r
(
n
a
m
e
l
i
s
t
)


 
 
 
 
 
 
 
 
c
l
.
v
i
s
i
t
(
t
o
p
)




 
 
 
 
 
 
 
 
#
 
E
x
e
c
u
t
e
 
t
h
e
 
m
o
d
i
f
i
e
d
 
A
S
T


 
 
 
 
 
 
 
 
t
e
m
p
 
=
 
{
}


 
 
 
 
 
 
 
 
e
x
e
c
(
c
o
m
p
i
l
e
(
t
o
p
,
'
'
,
'
e
x
e
c
'
)
,
 
t
e
m
p
,
 
t
e
m
p
)




 
 
 
 
 
 
 
 
#
 
P
u
l
l
 
o
u
t
 
t
h
e
 
m
o
d
i
f
i
e
d
 
c
o
d
e
 
o
b
j
e
c
t


 
 
 
 
 
 
 
 
f
u
n
c
.
_
_
c
o
d
e
_
_
 
=
 
t
e
m
p
[
f
u
n
c
.
_
_
n
a
m
e
_
_
]
.
_
_
c
o
d
e
_
_


 
 
 
 
 
 
 
 
r
e
t
u
r
n
 
f
u
n
c


 
 
 
 
r
e
t
u
r
n
 
l
o
w
e
r



为了使用这个代码，你可以像下面这样写：

In [ ]:
I
N
C
R
 
=
 
1


@
l
o
w
e
r
_
n
a
m
e
s
(
'
I
N
C
R
'
)


d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
n
 
-
=
 
I
N
C
R



装饰器会将 countdown() 函数重写为类似下面这样子：

In [ ]:
d
e
f
 
c
o
u
n
t
d
o
w
n
(
n
)
:


 
 
 
 
_
_
g
l
o
b
a
l
s
 
=
 
g
l
o
b
a
l
s
(
)


 
 
 
 
I
N
C
R
 
=
 
_
_
g
l
o
b
a
l
s
[
'
I
N
C
R
'
]


 
 
 
 
w
h
i
l
e
 
n
 
>
 
0
:


 
 
 
 
 
 
 
 
n
 
-
=
 
I
N
C
R



在性能测试中，它会让函数运行快20%

现在，你是不是想为你所有的函数都加上这个装饰器呢？或许不会。
但是，这却是对于一些高级技术比如AST操作、源码操作等等的一个很好的演示说明

本节受另外一个在 ActiveState 中处理Python字节码的章节的启示。
使用AST是一个更加高级点的技术，并且也更简单些。参考下面一节获得字节码的更多信息。

## 9.25 拆解Python字节码


### 问题


你想通过将你的代码反编译成低级的字节码来查看它底层的工作机制。

### 解决方案


dis 模块可以被用来输出任何Python函数的反编译结果。例如：

In [ ]:
def countdown(n):
while n > 0:
    print('T-minus', n)
    n -= 1
print('Blastoff!')
import dis
dis.dis(countdown)

### 讨论


当你想要知道你的程序底层的运行机制的时候，dis 模块是很有用的。比如如果你想试着理解性能特征。
被 dis() 函数解析的原始字节码如下所示：

In [ ]:
countdown.__code__.co_code

如果你想自己解释这段代码，你需要使用一些在 opcode 模块中定义的常量。例如：

In [ ]:
c = countdown.__code__.co_code
import opcode
opcode.opname[c[0]]
opcode.opname[c[0]]

In [ ]:
opcode.opname[c[3]]

奇怪的是，在 dis 模块中并没有函数让你以编程方式很容易的来处理字节码。
不过，下面的生成器函数可以将原始字节码序列转换成 opcodes 和参数。

In [ ]:
i
m
p
o
r
t
 
o
p
c
o
d
e




d
e
f
 
g
e
n
e
r
a
t
e
_
o
p
c
o
d
e
s
(
c
o
d
e
b
y
t
e
s
)
:


 
 
 
 
e
x
t
e
n
d
e
d
_
a
r
g
 
=
 
0


 
 
 
 
i
 
=
 
0


 
 
 
 
n
 
=
 
l
e
n
(
c
o
d
e
b
y
t
e
s
)


 
 
 
 
w
h
i
l
e
 
i
 
<
 
n
:


 
 
 
 
 
 
 
 
o
p
 
=
 
c
o
d
e
b
y
t
e
s
[
i
]


 
 
 
 
 
 
 
 
i
 
+
=
 
1


 
 
 
 
 
 
 
 
i
f
 
o
p
 
>
=
 
o
p
c
o
d
e
.
H
A
V
E
_
A
R
G
U
M
E
N
T
:


 
 
 
 
 
 
 
 
 
 
 
 
o
p
a
r
g
 
=
 
c
o
d
e
b
y
t
e
s
[
i
]
 
+
 
c
o
d
e
b
y
t
e
s
[
i
+
1
]
*
2
5
6
 
+
 
e
x
t
e
n
d
e
d
_
a
r
g


 
 
 
 
 
 
 
 
 
 
 
 
e
x
t
e
n
d
e
d
_
a
r
g
 
=
 
0


 
 
 
 
 
 
 
 
 
 
 
 
i
 
+
=
 
2


 
 
 
 
 
 
 
 
 
 
 
 
i
f
 
o
p
 
=
=
 
o
p
c
o
d
e
.
E
X
T
E
N
D
E
D
_
A
R
G
:


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
e
x
t
e
n
d
e
d
_
a
r
g
 
=
 
o
p
a
r
g
 
*
 
6
5
5
3
6


 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
c
o
n
t
i
n
u
e


 
 
 
 
 
 
 
 
e
l
s
e
:


 
 
 
 
 
 
 
 
 
 
 
 
o
p
a
r
g
 
=
 
N
o
n
e


 
 
 
 
 
 
 
 
y
i
e
l
d
 
(
o
p
,
 
o
p
a
r
g
)



使用方法如下：

In [ ]:
for op, oparg in generate_opcodes(countdown.__code__.co_code):
    print(op, opcode.opname[op], oparg)

这种方式很少有人知道，你可以利用它替换任何你想要替换的函数的原始字节码。
下面我们用一个示例来演示整个过程：

In [ ]:
def add(x, y):
    return x + y
c = add.__code__
c

In [ ]:
c.co_code

In [ ]:
# Make a completely new code object with bogus byte code
import types
newbytecode = b'xxxxxxx'
nc = types.CodeType(c.co_argcount, c.co_kwonlyargcount,
    c.co_nlocals, c.co_stacksize, c.co_flags, newbytecode, c.co_consts,
    c.co_names, c.co_varnames, c.co_filename, c.co_name,
    c.co_firstlineno, c.co_lnotab)
nc

In [ ]:
add.__code__ = nc
add(2,3)

你可以像这样耍大招让解释器奔溃。但是，对于编写更高级优化和元编程工具的程序员来讲，
他们可能真的需要重写字节码。本节最后的部分演示了这个是怎样做到的。你还可以参考另外一个类似的例子：
this code on ActiveState